<a href="https://colab.research.google.com/github/dejiandrew/nba-award-predictor/blob/deji/lightgbm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm *.csv
!rm *.pkl

rm: cannot remove '*.pkl': No such file or directory


In [ ]:
!gsutil -m cp -r gs://nba_award_predictor/nba_data/models/ag_models ./

Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F6/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/learner.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/metadata.json...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F7/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F4/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F1/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F5/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F8/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F2/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/ag_models/models/CatBoost_BAG_L1/S1F3/model.pkl...
Copying gs://nba_award_predictor/nba_data/models/a

In [ ]:
!pip install wget
!pip install autogluon
!pip install lightgbm
import pandas as pd
import wget
import duckdb
import lightgbm as lgb
import joblib
import numpy as np

In [ ]:
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly.csv')
features_overall_weekly_df = pd.read_csv('features-overall-weekly.csv')

In [ ]:
# Remove unwanted features
features_overall_weekly_df = features_overall_weekly_df.drop(columns=['league_pts_mean', 'league_pts_std',
       'league_ast_mean', 'league_ast_std', 'league_pm_mean', 'league_pm_std',
       'z_s_pts', 'z_s_ast', 'z_s_pm'])
features_overall_weekly_df

,player_id,full_name,team,season,week,week_start,conference,pow_conference,games_played_this_week,numMinutes,...,points_mean_season,points_std_season,assists_mean_season,assists_std_season,plusMinusPoints_mean_season,plusMinusPoints_std_season,z_pts,z_ast,z_pm,breakout_score
0,305,Robert Parish,Warriors,1979,1,1979-12-31,West,West,3,101.00,...,63.000000,7.874008,8.600000,3.382307,0.000000,0.000000,-0.254000,-1.655675,0.000000,-0.623703
1,76003,Kareem Abdul-Jabbar,Lakers,1979,1,1979-12-31,West,West,2,82.00,...,105.000000,25.674890,19.400000,7.200000,0.000000,0.000000,-1.986377,-0.750000,0.000000,-1.218188
2,76005,Tom Abernethy,Warriors,1979,1,1979-12-31,West,West,1,11.00,...,22.000000,13.190906,4.750000,2.680951,0.000000,0.000000,-1.667816,-1.771759,0.000000,-1.365436
3,76011,Alvan Adams,Suns,1979,1,1979-12-31,West,West,2,64.00,...,44.200000,27.095387,13.800000,7.884161,0.000000,0.000000,-0.634794,-0.862489,0.000000,-0.576144
4,76085,James Bailey,SuperSonics,1979,1,1979-12-31,West,West,4,41.00,...,18.000000,6.745369,0.750000,0.433013,0.000000,0.000000,-0.444750,2.886751,0.000000,0.643651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235264,1642942,Jahmai Mashack,Grizzlies,2025,51,2025-12-15,West,West,2,9.48,...,0.000000,0.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
235265,1642948,Ryan Nembhard,Mavericks,2025,51,2025-12-15,West,West,3,85.60,...,32.333333,23.976841,17.333333,12.472191,-3.333333,11.145502,-0.222437,0.454344,-0.239259,-0.022767
235266,1642949,Yanic Konan Niederhauser,Clippers,2025,51,2025-12-15,West,West,1,10.20,...,12.600000,10.800000,1.800000,2.400000,-12.000000,28.586710,-0.611111,-0.750000,0.944495,-0.341657
235267,1642954,Will Richard,Warriors,2025,51,2025-12-15,West,West,1,19.15,...,27.500000,15.392098,3.166667,1.572330,6.000000,25.252063,-0.487263,-2.013996,-0.198004,-0.887431


In [ ]:
features_overall_weekly_df.columns

Index(['player_id', 'full_name', 'team', 'season', 'week', 'week_start',
       'conference', 'pow_conference', 'games_played_this_week', 'numMinutes',
       'points', 'assists', 'blocks', 'steals', 'reboundsTotal',
       'reboundsDefensive', 'reboundsOffensive', 'fieldGoalsAttempted',
       'fieldGoalsMade', 'threePointersAttempted', 'threePointersMade',
       'freeThrowsAttempted', 'freeThrowsMade', 'turnovers', 'foulsPersonal',
       'plusMinusPoints', 'wins_this_week', 'wins_vs_team_with_all_nba_player',
       'is_win_vs_over_500', 'opponent_has_all_nba', 'avg_opp_score',
       'avg_opp_winrate_prior', 'avg_opp_wins_prior', 'avg_opp_losses_prior',
       'away_games_prior', 'away_losses_prior', 'away_win_streak_prior',
       'away_wins_prior', 'home_games_prior', 'home_losses_prior',
       'home_win_streak_prior', 'home_wins_prior', 'losses_prior',
       'wins_vs_over_500_prior', 'won_player_of_the_week',
       'all_star_this_season', 'mvp_this_season',
       'all_nba_f

In [ ]:
query = """
WITH CTE AS (
SELECT team, season, week, SUM(assists) AS team_assists
FROM features_overall_weekly_df
GROUP BY team, season, week
)
,CTE2 AS (

SELECT team, season, week, SUM(fieldGoalsMade) AS team_fg
FROM features_overall_weekly_df
GROUP BY team, season, week
)
, CTE3 AS (
SELECT CTE.team, CTE.season, CTE.week, team_assists, team_fg
FROM CTE
JOIN CTE2
ON CTE.team = CTE2.team AND CTE.season = CTE2.season AND CTE.week = CTE2.week
)
, CTE4 AS (

SELECT team, season, week, SUM(points)/team_gms AS team_ppg
FROM features_overall_weekly_df
GROUP BY team, season, week, team_gms
)

, CTE5 AS (
SELECT *
,team_pts/team_gms AS team_ppg
FROM features_overall_weekly_df a
JOIN CTE3 b
ON a.team=b.team AND a.season = b.season AND a.week=b.week
)

, CTE6 AS (
  SELECT *,
  CASE WHEN team_ppg > 186 THEN 115 ELSE team_ppg END AS team_ppg_fixed  -- If team_ppg shows as greater than 186, set it to 115
  FROM CTE5
)

SELECT *,
AVG(team_ppg_fixed) OVER(PARTITION BY season, week) AS lg_ppg
FROM CTE6

"""
features_overall_weekly_df = duckdb.query(query).df().drop(columns=['team_1','season_1','week_1','team_ppg'])
features_overall_weekly_df

# League-wide statistics (scalars)
lg_AST = features_overall_weekly_df['assists'].mean()
lg_FG = features_overall_weekly_df['fieldGoalsMade'].mean()
lg_FT = features_overall_weekly_df['freeThrowsMade'].mean()
lg_PF = features_overall_weekly_df['foulsPersonal'].mean()
lg_FTA = features_overall_weekly_df['freeThrowsAttempted'].mean()

factor = (2/3) - (0.5 * (lg_AST / lg_FG)) / (2 * (lg_FG / lg_FT))
vop = features_overall_weekly_df['points'].mean() / (features_overall_weekly_df['fieldGoalsAttempted'].mean() - features_overall_weekly_df['reboundsOffensive'].mean() + features_overall_weekly_df['turnovers'].mean() + 0.44 * lg_FTA)
drb_pct = (features_overall_weekly_df['reboundsTotal'].mean() - features_overall_weekly_df['reboundsOffensive'].mean()) / features_overall_weekly_df['reboundsTotal'].mean()

features_overall_weekly_df['u_per'] = (1 / features_overall_weekly_df['numMinutes']) * (
    features_overall_weekly_df['threePointersMade']
    + (2/3) * features_overall_weekly_df['assists']
    + (2 - factor * (features_overall_weekly_df['team_assists'] / features_overall_weekly_df['team_fg'])) * features_overall_weekly_df['fieldGoalsMade']
    + (features_overall_weekly_df['freeThrowsMade'] * 0.5 * (1 + (1 - (features_overall_weekly_df['team_assists'] / features_overall_weekly_df['team_fg'])) + (2/3) * (features_overall_weekly_df['team_assists'] / features_overall_weekly_df['team_fg'])))
    - vop * features_overall_weekly_df['turnovers']
    - vop * drb_pct * (features_overall_weekly_df['fieldGoalsAttempted'] - features_overall_weekly_df['fieldGoalsMade'])
    - vop * 0.44 * (0.44 + (0.56 * drb_pct)) * (features_overall_weekly_df['freeThrowsAttempted'] - features_overall_weekly_df['freeThrowsMade'])
    + vop * (1 - drb_pct) * (features_overall_weekly_df['reboundsTotal'] - features_overall_weekly_df['reboundsOffensive'])
    + vop * drb_pct * features_overall_weekly_df['reboundsOffensive']
    + vop * features_overall_weekly_df['steals']
    + vop * drb_pct * features_overall_weekly_df['blocks']
    - features_overall_weekly_df['foulsPersonal'] * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * vop)
)

# First, check for problematic values
print("Minutes == 0:", (features_overall_weekly_df['numMinutes'] == 0).sum())
print("team_fg == 0:", (features_overall_weekly_df['team_fg'] == 0).sum())
print("NaN values in key columns:")
print(features_overall_weekly_df[['numMinutes', 'team_assists', 'team_fg']].isna().sum())

# Filter to only calculate for valid rows
valid_mask = (
    (features_overall_weekly_df['numMinutes'] > 0) &
    (features_overall_weekly_df['team_fg'] > 0) &
    features_overall_weekly_df['numMinutes'].notna()
)

# Initialize u_per with NaN
features_overall_weekly_df['u_per'] = np.nan

# Calculate only for valid rows
lg_AST = features_overall_weekly_df['assists'].mean()
lg_FG = features_overall_weekly_df['fieldGoalsMade'].mean()
lg_FT = features_overall_weekly_df['freeThrowsMade'].mean()
lg_PF = features_overall_weekly_df['foulsPersonal'].mean()
lg_FTA = features_overall_weekly_df['freeThrowsAttempted'].mean()

factor = (2/3) - (0.5 * (lg_AST / lg_FG)) / (2 * (lg_FG / lg_FT))
vop = features_overall_weekly_df['points'].mean() / (features_overall_weekly_df['fieldGoalsAttempted'].mean() - features_overall_weekly_df['reboundsOffensive'].mean() + features_overall_weekly_df['turnovers'].mean() + 0.44 * lg_FTA)
drb_pct = (features_overall_weekly_df['reboundsTotal'].mean() - features_overall_weekly_df['reboundsOffensive'].mean()) / features_overall_weekly_df['reboundsTotal'].mean()

features_overall_weekly_df.loc[valid_mask, 'u_per'] = (1 / features_overall_weekly_df.loc[valid_mask, 'numMinutes']) * (
    features_overall_weekly_df.loc[valid_mask, 'threePointersMade']
    + (2/3) * features_overall_weekly_df.loc[valid_mask, 'assists']
    + (2 - factor * (features_overall_weekly_df.loc[valid_mask, 'team_assists'] / features_overall_weekly_df.loc[valid_mask, 'team_fg'])) * features_overall_weekly_df.loc[valid_mask, 'fieldGoalsMade']
    + (features_overall_weekly_df.loc[valid_mask, 'freeThrowsMade'] * 0.5 * (1 + (1 - (features_overall_weekly_df.loc[valid_mask, 'team_assists'] / features_overall_weekly_df.loc[valid_mask, 'team_fg'])) + (2/3) * (features_overall_weekly_df.loc[valid_mask, 'team_assists'] / features_overall_weekly_df.loc[valid_mask, 'team_fg'])))
    - vop * features_overall_weekly_df.loc[valid_mask, 'turnovers']
    - vop * drb_pct * (features_overall_weekly_df.loc[valid_mask, 'fieldGoalsAttempted'] - features_overall_weekly_df.loc[valid_mask, 'fieldGoalsMade'])
    - vop * 0.44 * (0.44 + (0.56 * drb_pct)) * (features_overall_weekly_df.loc[valid_mask, 'freeThrowsAttempted'] - features_overall_weekly_df.loc[valid_mask, 'freeThrowsMade'])
    + vop * (1 - drb_pct) * (features_overall_weekly_df.loc[valid_mask, 'reboundsTotal'] - features_overall_weekly_df.loc[valid_mask, 'reboundsOffensive'])
    + vop * drb_pct * features_overall_weekly_df.loc[valid_mask, 'reboundsOffensive']
    + vop * features_overall_weekly_df.loc[valid_mask, 'steals']
    + vop * drb_pct * features_overall_weekly_df.loc[valid_mask, 'blocks']
    - features_overall_weekly_df.loc[valid_mask, 'foulsPersonal'] * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * vop)
)

# # Check results
# print("\nAfter filtering:")
# print("inf values:", np.isinf(features_overall_weekly_df['u_per']).sum())
# print("NaN values:", features_overall_weekly_df['u_per'].isna().sum())
# print("Valid u_per values:", valid_mask.sum())

features_overall_weekly_df['pace_adjustment'] = 2 * features_overall_weekly_df['lg_ppg'] / (features_overall_weekly_df['team_ppg_fixed'] + features_overall_weekly_df['avg_opp_score'])
features_overall_weekly_df['a_per'] = features_overall_weekly_df['pace_adjustment'] * features_overall_weekly_df['u_per']
league_avg_aPER = features_overall_weekly_df['a_per'].mean()
features_overall_weekly_df['player_efficiency_rating'] = features_overall_weekly_df['a_per'] * (15 / league_avg_aPER)

# Check final results
print("Final PER distribution:")
print(features_overall_weekly_df['player_efficiency_rating'].describe())
print()
print(f"League average PER: {features_overall_weekly_df['player_efficiency_rating'].mean():.2f} (should be ~15.0)")

# Apply minimum minutes threshold (common in PER calculations)
min_minutes = 5  # or whatever threshold makes sense for your data

features_overall_weekly_df.loc[
    features_overall_weekly_df['numMinutes'] < min_minutes,
    'player_efficiency_rating'
] = np.nan

print(f"\nAfter {min_minutes} minute filter:")
print(features_overall_weekly_df['player_efficiency_rating'].describe())

features_overall_weekly_df = features_overall_weekly_df.drop(columns=['team_assists', 'team_fg','team_ppg_fixed', 'lg_ppg', 'u_per', 'pace_adjustment', 'a_per'])


Minutes == 0: 1250
team_fg == 0: 2
NaN values in key columns:
numMinutes      0
team_assists    0
team_fg         0
dtype: int64
Final PER distribution:
count    234017.000000
mean         15.000000
std          13.340741
min        -199.909466
25%           9.154771
50%          15.167629
75%          21.095042
max         684.869471
Name: player_efficiency_rating, dtype: float64

League average PER: 15.00 (should be ~15.0)

After 5 minute filter:
count    225023.000000
mean         15.208657
std          10.208857
min         -92.017766
25%           9.571464
50%          15.306903
75%          21.048558
max         160.646608
Name: player_efficiency_rating, dtype: float64


In [ ]:
# query = """
# WITH CTE AS (
# SELECT *,
# AVG(player_efficiency_rating) OVER(PARTITION BY team, season, week) as team_avg_PER

# FROM features_overall_weekly_df
# )

# SELECT * FROM CTE

# """

# features_overall_weekly_df = duckdb.query(query).df()
# features_overall_weekly_df

In [ ]:
# Only train on 2021+ seasons to account for modern scoring patterns.
#features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['season'] >= 2021]
#features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-12-22']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-12-15']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-12-08']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-12-01']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-11-24']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-11-17']
# features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-11-10']

In [ ]:
features_overall_weekly_df.columns

Index(['player_id', 'full_name', 'team', 'season', 'week', 'week_start',
       'conference', 'pow_conference', 'games_played_this_week', 'numMinutes',
       'points', 'assists', 'blocks', 'steals', 'reboundsTotal',
       'reboundsDefensive', 'reboundsOffensive', 'fieldGoalsAttempted',
       'fieldGoalsMade', 'threePointersAttempted', 'threePointersMade',
       'freeThrowsAttempted', 'freeThrowsMade', 'turnovers', 'foulsPersonal',
       'plusMinusPoints', 'wins_this_week', 'wins_vs_team_with_all_nba_player',
       'is_win_vs_over_500', 'opponent_has_all_nba', 'avg_opp_score',
       'avg_opp_winrate_prior', 'avg_opp_wins_prior', 'avg_opp_losses_prior',
       'away_games_prior', 'away_losses_prior', 'away_win_streak_prior',
       'away_wins_prior', 'home_games_prior', 'home_losses_prior',
       'home_win_streak_prior', 'home_wins_prior', 'losses_prior',
       'wins_vs_over_500_prior', 'won_player_of_the_week',
       'all_star_this_season', 'mvp_this_season',
       'all_nba_f

In [ ]:
# View the class imbalance (99.4% positive class, 0.6% negative class)
print(features_overall_weekly_df['won_player_of_the_week'].value_counts())
print(features_overall_weekly_df['won_player_of_the_week'].value_counts(normalize=True))

won_player_of_the_week
0    233872
1      1397
Name: count, dtype: int64
won_player_of_the_week
0    0.994062
1    0.005938
Name: proportion, dtype: float64


In [ ]:
# Drop columns that contain the answer. Also drop mvp, all-star, all-nba because they are from the future when we try to do inference during season
leakage_cols = ['pow_player_id', 'player_of_the_week', 'pow_conference',
                'all_star_this_season', 'mvp_this_season',
                'all_nba_first_team_this_season', 'all_nba_second_team_this_season',
                'all_nba_third_team_this_season']

# Set aside identifier columns
id_cols = ['player_id', 'full_name', 'team', 'season', 'week', 'week_start', 'conference']

features_overall_weekly_df_clean = features_overall_weekly_df.drop(columns=leakage_cols + id_cols)
df_encoded = pd.get_dummies(features_overall_weekly_df_clean, drop_first=True)
df_encoded

,games_played_this_week,numMinutes,points,assists,blocks,steals,reboundsTotal,reboundsDefensive,reboundsOffensive,fieldGoalsAttempted,...,points_std_season,assists_mean_season,assists_std_season,plusMinusPoints_mean_season,plusMinusPoints_std_season,z_pts,z_ast,z_pm,breakout_score,player_efficiency_rating
0,4,142.00,83.0,11.0,11.0,2.0,43.0,32.0,11.0,65.0,...,12.666667,6.666667,3.559026,0.000000,0.000000,2.157895,1.217562,0.000000,1.444216,22.120472
1,3,84.00,50.0,17.0,0.0,1.0,13.0,2.0,1.0,36.0,...,29.111853,16.250000,7.822244,0.000000,0.000000,-0.841582,0.095880,0.000000,-0.392027,20.341994
2,3,106.00,83.0,19.0,16.0,6.0,21.0,14.0,7.0,51.0,...,25.521233,17.444444,6.075655,0.000000,0.000000,-0.444075,0.256031,0.000000,-0.145228,40.606844
3,3,23.00,0.0,3.0,1.0,0.0,4.0,4.0,0.0,2.0,...,13.917166,4.000000,2.915476,0.000000,0.000000,-1.095769,-0.342997,0.000000,-0.650784,2.127043
4,3,92.00,53.0,10.0,7.0,4.0,28.0,4.0,2.0,40.0,...,23.060307,13.777778,6.778688,0.000000,0.000000,0.404736,-0.557302,0.000000,0.035177,24.934914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235264,1,0.25,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.947457,1.000000,0.707107,-5.625000,9.745993,-0.424094,-1.414214,0.474554,-0.541400,NaN
235265,1,8.46,4.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,...,18.751568,1.538462,1.946017,-8.461538,13.793061,-0.713786,-0.790569,0.613463,-0.471371,16.732762
235266,3,83.90,33.0,4.0,0.0,3.0,6.0,6.0,0.0,34.0,...,10.443238,2.357143,1.797107,-10.214286,14.173308,1.600489,0.914168,-2.524867,0.569521,9.509411
235267,3,53.76,17.0,1.0,2.0,1.0,5.0,1.0,4.0,14.0,...,1.160577,0.142857,0.349927,-2.571429,5.948761,14.032430,2.449490,-0.072044,7.736653,10.336200


In [ ]:
from sklearn.model_selection import GroupShuffleSplit

X = df_encoded.drop(columns=['won_player_of_the_week'])
y = df_encoded['won_player_of_the_week']
identifiers = features_overall_weekly_df[id_cols]

# Create groups for each season-week-conference combination
groups = (features_overall_weekly_df['season'].astype(str) + '_' +
          features_overall_weekly_df['week'].astype(str) + '_' +
          features_overall_weekly_df['conference'].astype(str))

# Use GroupShuffleSplit instead of train_test_split
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

# Split using the indices
X_train = X.iloc[train_idx]
X_test = X.iloc[test_idx]
y_train = y.iloc[train_idx]
y_test = y.iloc[test_idx]
id_train = identifiers.iloc[train_idx]
id_test = identifiers.iloc[test_idx]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Positive class in y_train: {100 * round(y_train.mean(),4)}%")
print(f"Positive class in y_test: {100 * round(y_test.mean(),4)}%")

Training samples: 187552
Test samples: 47717
Positive class in y_train: 0.6%
Positive class in y_test: 0.59%


In [ ]:
# Train LightGBM model
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    num_leaves=31,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# Train LightGBM model
lgb_model.fit(X_train, y_train)

# Get predictions
y_pred_proba = lgb_model.predict_proba(X_test)[:, 1]

In [ ]:
evaluation_df = id_test.copy()
evaluation_df['actual_won'] = y_test.values
evaluation_df['pred_proba'] = y_pred_proba
evaluation_df

,player_id,full_name,team,season,week,week_start,conference,actual_won,pred_proba
1455,3,Grant Long,Heat,1988,10,1989-03-06,East,0,0.000757
1456,22,Rik Smits,Pacers,1988,10,1989-03-06,East,0,0.003224
1457,23,Dennis Rodman,Pistons,1988,10,1989-03-06,East,0,0.001355
1458,52,Micheal Williams,Pistons,1988,10,1989-03-06,East,0,0.000085
1459,73,Hot Rod Williams,Cavaliers,1988,10,1989-03-06,East,0,0.001912
...,...,...,...,...,...,...,...,...,...
235262,1642382,Branden Carlson,Thunder,2024,12,2025-03-17,West,0,0.000063
235263,1642384,Isaiah Crawford,Kings,2024,12,2025-03-17,West,0,0.000124
235265,1642403,Isaac Jones,Kings,2024,12,2025-03-17,West,0,0.000116
235267,1642461,Spencer Jones,Nuggets,2024,12,2025-03-17,West,0,0.000035


In [ ]:
actual_winners_df = evaluation_df[evaluation_df['actual_won'] == 1]
actual_winners_df.head()

,player_id,full_name,team,season,week,week_start,conference,actual_won,pred_proba
1593,78318,Isiah Thomas,Pistons,1988,10,1989-03-06,East,1,0.975065
3529,686,Antonio McDyess,Nuggets,1998,8,1999-02-22,West,1,0.972441
3923,406,Shaquille O'Neal,Lakers,1999,8,2000-02-21,West,1,0.988290
4252,1495,Tim Duncan,Spurs,1999,13,2000-03-27,West,1,0.984656
5414,947,Allen Iverson,76ers,2002,48,2002-11-25,East,1,0.979060


In [ ]:
print(f"Total samples in evaluation_df: {len(evaluation_df)}")
print(f"Actual winners: {len(actual_winners_df)}")

Total samples in evaluation_df: 47717
Actual winners: 280


In [ ]:
# How many season,conference,week combinations were there in actual_winners_df? This is our denominator for accuracy.

query = """
SELECT COUNT(DISTINCT(season,conference,week)) AS total_season_conference_weeks_we_couldve_predicted
FROM actual_winners_df
"""

total_season_conference_weeks_we_couldve_predicted = duckdb.query(query).df()['total_season_conference_weeks_we_couldve_predicted'].sum()
total_season_conference_weeks_we_couldve_predicted

np.int64(277)

In [ ]:
# How many of the actual winners did we predict?
# Aka how many of the actual winners did our model assign a top-k probability for that [season,conference,week] combo?
for k in [1, 2, 3, 4, 5, 10]:
  query = f"""

  WITH Highest_Probabilities AS (
  SELECT *
  ,RANK() OVER(PARTITION BY season, conference, week ORDER BY pred_proba DESC) AS rnk
  FROM evaluation_df
  )
  ,
  Actual_Winners AS (
    SELECT * FROM evaluation_df
    WHERE actual_won = 1
  )


  SELECT * FROM Highest_Probabilities
  JOIN Actual_Winners
  ON
  Highest_Probabilities.season = Actual_Winners.season
  AND
  Highest_Probabilities.week = Actual_Winners.week
  AND
  Highest_Probabilities.conference = Actual_Winners.conference
  WHERE rnk <= {k}
  AND Highest_Probabilities.pred_proba = Actual_Winners.pred_proba
  """
  duckdb.query(query).df()
  correct_predictions_df = duckdb.query(query).df()
  total_correct_predictions = correct_predictions_df["actual_won"].sum()
  print(f"Top-{k} accuracy:", round(100*total_correct_predictions/total_season_conference_weeks_we_couldve_predicted,2),"%")

Top-1 accuracy: 56.32 %
Top-2 accuracy: 73.65 %
Top-3 accuracy: 81.59 %
Top-4 accuracy: 86.28 %
Top-5 accuracy: 89.89 %
Top-10 accuracy: 97.11 %


In [ ]:
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

feature_importance['importance_pct'] = (feature_importance['importance'] / feature_importance['importance'].sum() * 100)

print("LightGBM Top 20 features:")
print(feature_importance.head(20)[['feature','importance_pct']])

LightGBM Top 20 features:
                     feature  importance_pct
54  player_efficiency_rating        8.517034
36                  team_pts        5.711423
2                     points        5.010020
18            wins_this_week        4.342017
44        points_mean_season        3.540414
1                 numMinutes        2.939212
39                  team_stl        2.839011
10            fieldGoalsMade        2.705411
46       assists_mean_season        2.371409
17           plusMinusPoints        2.304609
15                 turnovers        2.304609
9        fieldGoalsAttempted        2.237809
6              reboundsTotal        2.237809
53            breakout_score        2.171009
41      fieldGoalsPercentage        2.137609
3                    assists        2.037408
13       freeThrowsAttempted        2.037408
37                  team_ast        2.004008
38                  team_blk        2.004008
22             avg_opp_score        1.937208


In [ ]:
# import joblib

# # Save model
# joblib.dump(lgb_model, 'lightgbm_potw_model.pkl')

# # Save feature names
# joblib.dump(X_train.columns.tolist(), 'lightgbm_model_features.pkl')

In [ ]:
print("\nTraining final model on 100% of data for deployment...")
final_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    num_leaves=31,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

final_model.fit(X, y)  # ALL data

# 5. Save the final model
joblib.dump(final_model, 'lightgbm_potw_model.pkl')
joblib.dump(X.columns.tolist(), 'lightgbm_model_features.pkl')


Training final model on 100% of data for deployment...


['lightgbm_model_features.pkl']

In [ ]:
# Feature importance from final model
final_feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

final_feature_importance['importance_pct'] = (final_feature_importance['importance'] / final_feature_importance['importance'].sum() * 100)

print("Final Model Top 20 features:")
print(final_feature_importance.head(20)[['feature','importance_pct']])

Final Model Top 20 features:
                       feature  importance_pct
54    player_efficiency_rating        9.103034
36                    team_pts        6.535512
2                       points        5.468489
18              wins_this_week        5.135045
1                   numMinutes        3.501167
44          points_mean_season        3.401134
10              fieldGoalsMade        2.500834
39                    team_stl        2.467489
50                       z_pts        2.300767
7            reboundsDefensive        2.267422
37                    team_ast        2.200734
3                      assists        2.100700
53              breakout_score        2.100700
17             plusMinusPoints        2.034011
9          fieldGoalsAttempted        2.034011
46         assists_mean_season        2.000667
13         freeThrowsAttempted        2.000667
15                   turnovers        1.967322
6                reboundsTotal        1.967322
49  plusMinusPoints_std_season 

**The below steps will use the above model for inference.**

In [ ]:
def get_potw_predictions(week_start: str):
  """
  get_potw_predictions() does inference to predict POW.
  week_start must be a Monday in 'yyyy-mm-dd' format
  """
  import duckdb
  import joblib
  import wget

  # features-overall-weekly-for-inference.csv is calculated in a separate script that runs within the daily production pipeline
  # features-overall-weekly-for-inference.csv is needed to make predictions while the current week is in-progress (i.e. no POW announced yet)
  wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly-for-inference.csv')

  # Read in model & its feature names
  model = joblib.load('lightgbm_potw_model.pkl')
  feature_names = joblib.load('lightgbm_model_features.pkl')

  # Read in player-week stats
  overall_weekly_agg_df = pd.read_csv('features-overall-weekly-for-inference.csv')

  # Remove unwanted features
  overall_weekly_agg_df = overall_weekly_agg_df.drop(columns=['league_pts_mean', 'league_pts_std',
       'league_ast_mean', 'league_ast_std', 'league_pm_mean', 'league_pm_std',
       'z_s_pts', 'z_s_ast', 'z_s_pm'])


##############################################################
##############################################################

  query = """
  WITH CTE AS (
  SELECT team, season, week, SUM(assists) AS team_assists
  FROM overall_weekly_agg_df
  GROUP BY team, season, week
  )
  ,CTE2 AS (

  SELECT team, season, week, SUM(fieldGoalsMade) AS team_fg
  FROM overall_weekly_agg_df
  GROUP BY team, season, week
  )
  , CTE3 AS (
  SELECT CTE.team, CTE.season, CTE.week, team_assists, team_fg
  FROM CTE
  JOIN CTE2
  ON CTE.team = CTE2.team AND CTE.season = CTE2.season AND CTE.week = CTE2.week
  )
  , CTE4 AS (

  SELECT team, season, week, SUM(points)/team_gms AS team_ppg
  FROM overall_weekly_agg_df
  GROUP BY team, season, week, team_gms
  )

  , CTE5 AS (
  SELECT *
  ,team_pts/team_gms AS team_ppg
  FROM overall_weekly_agg_df a
  JOIN CTE3 b
  ON a.team=b.team AND a.season = b.season AND a.week=b.week
  )

  , CTE6 AS (
    SELECT *,
    CASE WHEN team_ppg > 186 THEN 115 ELSE team_ppg END AS team_ppg_fixed  -- If team_ppg shows as greater than 186, set it to 115
    FROM CTE5
  )

  SELECT *,
  AVG(team_ppg_fixed) OVER(PARTITION BY season, week) AS lg_ppg
  FROM CTE6

  """
  overall_weekly_agg_df = duckdb.query(query).df().drop(columns=['team_1','season_1','week_1','team_ppg'])
  overall_weekly_agg_df

  # League-wide statistics (scalars)
  lg_AST = overall_weekly_agg_df['assists'].mean()
  lg_FG = overall_weekly_agg_df['fieldGoalsMade'].mean()
  lg_FT = overall_weekly_agg_df['freeThrowsMade'].mean()
  lg_PF = overall_weekly_agg_df['foulsPersonal'].mean()
  lg_FTA = overall_weekly_agg_df['freeThrowsAttempted'].mean()

  factor = (2/3) - (0.5 * (lg_AST / lg_FG)) / (2 * (lg_FG / lg_FT))
  vop = overall_weekly_agg_df['points'].mean() / (overall_weekly_agg_df['fieldGoalsAttempted'].mean() - overall_weekly_agg_df['reboundsOffensive'].mean() + overall_weekly_agg_df['turnovers'].mean() + 0.44 * lg_FTA)
  drb_pct = (overall_weekly_agg_df['reboundsTotal'].mean() - overall_weekly_agg_df['reboundsOffensive'].mean()) / overall_weekly_agg_df['reboundsTotal'].mean()

  overall_weekly_agg_df['u_per'] = (1 / overall_weekly_agg_df['numMinutes']) * (
      overall_weekly_agg_df['threePointersMade']
      + (2/3) * overall_weekly_agg_df['assists']
      + (2 - factor * (overall_weekly_agg_df['team_assists'] / overall_weekly_agg_df['team_fg'])) * overall_weekly_agg_df['fieldGoalsMade']
      + (overall_weekly_agg_df['freeThrowsMade'] * 0.5 * (1 + (1 - (overall_weekly_agg_df['team_assists'] / overall_weekly_agg_df['team_fg'])) + (2/3) * (overall_weekly_agg_df['team_assists'] / overall_weekly_agg_df['team_fg'])))
      - vop * overall_weekly_agg_df['turnovers']
      - vop * drb_pct * (overall_weekly_agg_df['fieldGoalsAttempted'] - overall_weekly_agg_df['fieldGoalsMade'])
      - vop * 0.44 * (0.44 + (0.56 * drb_pct)) * (overall_weekly_agg_df['freeThrowsAttempted'] - overall_weekly_agg_df['freeThrowsMade'])
      + vop * (1 - drb_pct) * (overall_weekly_agg_df['reboundsTotal'] - overall_weekly_agg_df['reboundsOffensive'])
      + vop * drb_pct * overall_weekly_agg_df['reboundsOffensive']
      + vop * overall_weekly_agg_df['steals']
      + vop * drb_pct * overall_weekly_agg_df['blocks']
      - overall_weekly_agg_df['foulsPersonal'] * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * vop)
  )


  # Filter to only calculate for valid rows
  valid_mask = (
      (overall_weekly_agg_df['numMinutes'] > 0) &
      (overall_weekly_agg_df['team_fg'] > 0) &
      overall_weekly_agg_df['numMinutes'].notna()
  )

  # Initialize u_per with NaN
  overall_weekly_agg_df['u_per'] = np.nan

  # Calculate only for valid rows
  lg_AST = overall_weekly_agg_df['assists'].mean()
  lg_FG = overall_weekly_agg_df['fieldGoalsMade'].mean()
  lg_FT = overall_weekly_agg_df['freeThrowsMade'].mean()
  lg_PF = overall_weekly_agg_df['foulsPersonal'].mean()
  lg_FTA = overall_weekly_agg_df['freeThrowsAttempted'].mean()

  factor = (2/3) - (0.5 * (lg_AST / lg_FG)) / (2 * (lg_FG / lg_FT))
  vop = overall_weekly_agg_df['points'].mean() / (overall_weekly_agg_df['fieldGoalsAttempted'].mean() - overall_weekly_agg_df['reboundsOffensive'].mean() + overall_weekly_agg_df['turnovers'].mean() + 0.44 * lg_FTA)
  drb_pct = (overall_weekly_agg_df['reboundsTotal'].mean() - overall_weekly_agg_df['reboundsOffensive'].mean()) / overall_weekly_agg_df['reboundsTotal'].mean()

  overall_weekly_agg_df.loc[valid_mask, 'u_per'] = (1 / overall_weekly_agg_df.loc[valid_mask, 'numMinutes']) * (
      overall_weekly_agg_df.loc[valid_mask, 'threePointersMade']
      + (2/3) * overall_weekly_agg_df.loc[valid_mask, 'assists']
      + (2 - factor * (overall_weekly_agg_df.loc[valid_mask, 'team_assists'] / overall_weekly_agg_df.loc[valid_mask, 'team_fg'])) * overall_weekly_agg_df.loc[valid_mask, 'fieldGoalsMade']
      + (overall_weekly_agg_df.loc[valid_mask, 'freeThrowsMade'] * 0.5 * (1 + (1 - (overall_weekly_agg_df.loc[valid_mask, 'team_assists'] / overall_weekly_agg_df.loc[valid_mask, 'team_fg'])) + (2/3) * (overall_weekly_agg_df.loc[valid_mask, 'team_assists'] / overall_weekly_agg_df.loc[valid_mask, 'team_fg'])))
      - vop * overall_weekly_agg_df.loc[valid_mask, 'turnovers']
      - vop * drb_pct * (overall_weekly_agg_df.loc[valid_mask, 'fieldGoalsAttempted'] - overall_weekly_agg_df.loc[valid_mask, 'fieldGoalsMade'])
      - vop * 0.44 * (0.44 + (0.56 * drb_pct)) * (overall_weekly_agg_df.loc[valid_mask, 'freeThrowsAttempted'] - overall_weekly_agg_df.loc[valid_mask, 'freeThrowsMade'])
      + vop * (1 - drb_pct) * (overall_weekly_agg_df.loc[valid_mask, 'reboundsTotal'] - overall_weekly_agg_df.loc[valid_mask, 'reboundsOffensive'])
      + vop * drb_pct * overall_weekly_agg_df.loc[valid_mask, 'reboundsOffensive']
      + vop * overall_weekly_agg_df.loc[valid_mask, 'steals']
      + vop * drb_pct * overall_weekly_agg_df.loc[valid_mask, 'blocks']
      - overall_weekly_agg_df.loc[valid_mask, 'foulsPersonal'] * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * vop)
  )


  overall_weekly_agg_df['pace_adjustment'] = 2 * overall_weekly_agg_df['lg_ppg'] / (overall_weekly_agg_df['team_ppg_fixed'] + overall_weekly_agg_df['avg_opp_score'])
  overall_weekly_agg_df['a_per'] = overall_weekly_agg_df['pace_adjustment'] * overall_weekly_agg_df['u_per']
  league_avg_aPER = overall_weekly_agg_df['a_per'].mean()
  overall_weekly_agg_df['player_efficiency_rating'] = overall_weekly_agg_df['a_per'] * (15 / league_avg_aPER)


  # Apply minimum minutes threshold (common in PER calculations)
  min_minutes = 5  # or whatever threshold makes sense for your data

  overall_weekly_agg_df.loc[
      overall_weekly_agg_df['numMinutes'] < min_minutes,
      'player_efficiency_rating'
  ] = np.nan


  overall_weekly_agg_df = overall_weekly_agg_df.drop(columns=['team_assists', 'team_fg','team_ppg_fixed', 'lg_ppg', 'u_per', 'pace_adjustment', 'a_per'])


################################################################
################################################################

  # Query all players who played for the input week
  query = f"""
  SELECT * FROM overall_weekly_agg_df
  WHERE week_start = '{week_start}'
  """
  df = duckdb.query(query).df()

  #Keep player info separate
  player_info = df[['player_id', 'full_name', 'conference', 'season', 'week']]
  X = df[feature_names]
  probabilities = model.predict_proba(X)[:, 1]

  # Attach probabilities to the player identities
  results = player_info.copy()
  results['probability'] = probabilities

  # Now partition by season, week, and conference and return top 5 for each conference
  results
  query = f"""
  WITH CTE AS (SELECT *,
  RANK() OVER(PARTITION BY season, week, conference ORDER BY probability DESC) AS rank
  FROM results
  )

  ,CTE2 AS (
  SELECT * FROM CTE WHERE conference = 'East' AND rank <= 5
  UNION ALL
  SELECT * FROM CTE WHERE conference = 'West' AND rank <= 5

  )
  SELECT
  '{week_start}' AS week_start
  ,rank
  ,full_name as name
  ,conference
  ,probability
  FROM CTE2
  """

  top_five_per_conference = duckdb.query(query).df()

  return top_five_per_conference

get_potw_predictions('2025-12-22')

,week_start,rank,name,conference,probability
0,2025-12-22,1,Karl-Anthony Towns,East,0.656866
1,2025-12-22,2,Kevin Porter Jr.,East,0.520688
2,2025-12-22,3,Jaylen Brown,East,0.466286
3,2025-12-22,4,Scottie Barnes,East,0.392603
4,2025-12-22,5,Nic Claxton,East,0.245099
5,2025-12-22,1,Kawhi Leonard,West,0.603993
6,2025-12-22,2,Stephon Castle,West,0.585991
7,2025-12-22,3,Amen Thompson,West,0.482635
8,2025-12-22,4,LeBron James,West,0.451301
9,2025-12-22,5,Julius Randle,West,0.380631


AutoGluon Model

In [ ]:
# def get_potw_predictions(week_start: str):
#   """
#   get_potw_predictions() does inference to predict POW.
#   week_start must be a Monday in 'yyyy-mm-dd' format
#   """
#   import duckdb
#   import joblib
#   import wget
#   import pandas as pd
#   from autogluon.tabular import TabularPredictor

#   # features-overall-weekly-for-inference.csv is calculated in a separate script that runs within the daily production pipeline
#   # features-overall-weekly-for-inference.csv is needed to make predictions while the current week is in-progress (i.e. no POW announced yet)
#   wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly-for-inference.csv')
#   wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/autogluon_model_features.pkl')

#   # Read in model (load the full predictor, not just the pkl)
#   predictor = TabularPredictor.load('./ag_models')
#   feature_names = joblib.load('autogluon_model_features.pkl')

#   # Read in player-week stats
#   overall_weekly_agg_df = pd.read_csv('features-overall-weekly-for-inference.csv')

#   # Remove unwanted features
#   overall_weekly_agg_df = overall_weekly_agg_df.drop(columns=['league_pts_mean', 'league_pts_std',
#        'league_ast_mean', 'league_ast_std', 'league_pm_mean', 'league_pm_std',
#        'z_s_pts', 'z_s_ast', 'z_s_pm'])

#   # Query all players who played for the input week
#   query = f"""
#   SELECT * FROM overall_weekly_agg_df
#   WHERE week_start = '{week_start}'
#   """
#   df = duckdb.query(query).df()

#   #Keep player info separate
#   player_info = df[['player_id', 'full_name', 'conference', 'season', 'week']]
#   X = df[feature_names]

#   # Use predictor.predict_proba instead
#   probabilities = predictor.predict_proba(X)[1].values  # Get probability of class 1

#   # Attach probabilities to the player identities
#   results = player_info.copy()
#   results['probability'] = probabilities

#   # Now partition by season, week, and conference and return top 5 for each conference
#   query = f"""
#   WITH CTE AS (SELECT *,
#   RANK() OVER(PARTITION BY season, week, conference ORDER BY probability DESC) AS rank
#   FROM results
#   )

#   ,CTE2 AS (
#   SELECT * FROM CTE WHERE conference = 'East' AND rank <= 5
#   UNION ALL
#   SELECT * FROM CTE WHERE conference = 'West' AND rank <= 5

#   )
#   SELECT
#   '{week_start}' AS week_start
#   ,rank
#   ,full_name as name
#   ,conference
#   ,probability
#   FROM CTE2
#   """

#   top_five_per_conference = duckdb.query(query).df()

#   return top_five_per_conference

# get_potw_predictions('2025-12-15')

Pick out a single model from autogluon and see results e.g. "LightGBM_BAG_L1"

In [ ]:
# def get_potw_predictions(week_start: str):
#   """
#   get_potw_predictions() does inference to predict POW.
#   week_start must be a Monday in 'yyyy-mm-dd' format
#   """
#   import duckdb
#   import joblib
#   import wget
#   import pandas as pd
#   from autogluon.tabular import TabularPredictor

#   # features-overall-weekly-for-inference.csv is calculated in a separate script that runs within the daily production pipeline
#   # features-overall-weekly-for-inference.csv is needed to make predictions while the current week is in-progress (i.e. no POW announced yet)
#   #wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly-for-inference.csv')
#   #wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/autogluon_model_features.pkl')

#   # Read in model (load the full predictor, not just the pkl)
#   predictor = TabularPredictor.load('./ag_models')
#   feature_names = joblib.load('autogluon_model_features.pkl')

#   # Read in player-week stats
#   overall_weekly_agg_df = pd.read_csv('features-overall-weekly-for-inference.csv')

#   # Remove unwanted features
#   overall_weekly_agg_df = overall_weekly_agg_df.drop(columns=['league_pts_mean', 'league_pts_std',
#        'league_ast_mean', 'league_ast_std', 'league_pm_mean', 'league_pm_std',
#        'z_s_pts', 'z_s_ast', 'z_s_pm'])

#   # Query all players who played for the input week
#   query = f"""
#   SELECT * FROM overall_weekly_agg_df
#   WHERE week_start = '{week_start}'
#   """
#   df = duckdb.query(query).df()

#   #Keep player info separate
#   player_info = df[['player_id', 'full_name', 'conference', 'season', 'week']]
#   X = df[feature_names]

#   # Use predictor.predict_proba with model='LightGBM_BAG_L1'
#   probabilities = predictor.predict_proba(X, model='LightGBM_BAG_L1')[1].values  # Get probability of class 1

#   # Attach probabilities to the player identities
#   results = player_info.copy()
#   results['probability'] = probabilities

#   # Now partition by season, week, and conference and return top 5 for each conference
#   query = f"""
#   WITH CTE AS (SELECT *,
#   RANK() OVER(PARTITION BY season, week, conference ORDER BY probability DESC) AS rank
#   FROM results
#   )

#   ,CTE2 AS (
#   SELECT * FROM CTE WHERE conference = 'East' AND rank <= 5
#   UNION ALL
#   SELECT * FROM CTE WHERE conference = 'West' AND rank <= 5

#   )
#   SELECT
#   '{week_start}' AS week_start
#   ,rank
#   ,full_name as name
#   ,conference
#   ,probability
#   FROM CTE2
#   """

#   top_five_per_conference = duckdb.query(query).df()

#   return top_five_per_conference

# get_potw_predictions('2025-12-15')

In [1]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=0df2cff88d4897554b87ad72bb37dfa6a834b6ca109dc9a51e70d21766220945
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [2]:
import wget
import pandas as pd
import duckdb
import numpy as np

In [3]:
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly.csv')

'features-overall-weekly.csv'

In [4]:
features_overall_df = pd.read_csv('features-overall.csv')
features_overall_weekly_df = pd.read_csv('features-overall-weekly.csv')

In [5]:
features_overall_df = features_overall_df[['gameId', 'gameDate', 'team','full_name','opponent',
                                           'home', 'team_score', 'opp_score','points', 'fieldGoalsAttempted',
                                           'fieldGoalsMade','win']]

query = """
WITH combined_teams AS (
SELECT DISTINCT

a.gameId,
a.gameDate,
a.team,
a.full_name,
a.opponent,
a.home,
a.team_score,
a.opp_score,
a.points,
a.fieldGoalsAttempted,
a.fieldGoalsMade,
a.win,


b.full_name as opp_player_full_name,
b.fieldGoalsAttempted AS opp_player_fieldGoalsAttempted,
b.fieldGoalsMade AS opp_player_fieldGoalsMade

FROM
features_overall_df a
JOIN features_overall_df b
ON a.gameId = b.gameId

)
, home_fgs_by_player AS (
SELECT DISTINCT gameId, gameDate, team,full_name,fieldGoalsAttempted
FROM combined_teams
WHERE home = 1
)
, home_fg_aggregate AS (

  SELECT gameId, gameDate, team as home_team, SUM(fieldGoalsAttempted) as home_team_fga
  FROM home_fgs_by_player
  GROUP BY gameId, gameDate, team
)

, away_fgs_by_player AS (
SELECT DISTINCT gameId, gameDate, team,full_name,fieldGoalsAttempted
FROM combined_teams
WHERE home = 0
)
, away_fg_aggregate AS (

  SELECT gameId, gameDate, team as away_team, SUM(fieldGoalsAttempted) as away_team_fga
  FROM away_fgs_by_player
  GROUP BY gameId, gameDate, team
)

, home_away_fg_comparison AS (

SELECT home_fg_aggregate.gameId,
home_fg_aggregate.gameDate,
home_team,
CAST(home_team_fga AS INT) as home_team_fga,
away_team,
CAST(away_team_fga AS INT) as away_team_fga
FROM home_fg_aggregate
JOIN away_fg_aggregate
ON home_fg_aggregate.gameId = away_fg_aggregate.gameId

)
, combine_fg_aggregate_with_base_table AS (
SELECT *
FROM home_away_fg_comparison
JOIN features_overall_df
ON home_away_fg_comparison.gameId = features_overall_df.gameId
)

SELECT DISTINCT

gameId
,CAST(gameDate AS DATE) AS gameDate
,team
,opponent
,CASE
  WHEN team = home_team THEN home_team_fga
  WHEN team = away_team THEN away_team_fga
  ELSE NULL
END AS team_fga
,CASE
  WHEN team = home_team THEN away_team_fga
  WHEN team = away_team THEN home_team_fga
  ELSE NULL
END AS opp_team_fga
,team_score AS team_pts
,opp_score AS opp_team_pts
FROM combine_fg_aggregate_with_base_table

"""

field_goal_features_df = duckdb.query(query).df()
field_goal_features_df['week_start'] = field_goal_features_df['gameDate'] - field_goal_features_df['gameDate'].dt.weekday.astype('timedelta64[D]')
field_goal_features_df

query = """

SELECT week_start
,team
,CAST(SUM(team_fga) AS INT) AS team_fga
,CAST(SUM(opp_team_fga) AS INT) AS opp_team_fga
,CAST(SUM(team_pts) AS INT) AS team_pts
,CAST(SUM(opp_team_pts) AS INT) AS opp_team_pts
FROM field_goal_features_df
GROUP BY week_start, team
"""


field_goal_features_weekly_agg_df = duckdb.query(query).df()

features_overall_weekly_df['week_start'] = pd.to_datetime(features_overall_weekly_df['week_start'])

features_overall_weekly_df = features_overall_weekly_df.merge(field_goal_features_weekly_agg_df,on=['team','week_start'],how='left').drop(columns='team_pts_x').rename(columns={'team_pts_y':'team_pts'})

features_overall_weekly_df['team_fga'] = features_overall_weekly_df['team_fga'].fillna(features_overall_weekly_df['team_fga'].mean())
features_overall_weekly_df['opp_team_fga'] = features_overall_weekly_df['opp_team_fga'].fillna(features_overall_weekly_df['opp_team_fga'].mean())
features_overall_weekly_df['team_pts'] = features_overall_weekly_df['team_pts'].fillna(features_overall_weekly_df['team_pts'].mean())
features_overall_weekly_df['opp_team_pts'] = features_overall_weekly_df['opp_team_pts'].fillna(features_overall_weekly_df['opp_team_pts'].mean())

features_overall_weekly_df = features_overall_weekly_df.drop(columns=['league_pts_mean', 'league_pts_std',
       'league_ast_mean', 'league_ast_std', 'league_pm_mean', 'league_pm_std',
       'z_s_pts', 'z_s_ast', 'z_s_pm'])
features_overall_weekly_df


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,player_id,full_name,team,season,week,week_start,conference,pow_conference,games_played_this_week,numMinutes,...,plusMinusPoints_mean_season,plusMinusPoints_std_season,z_pts,z_ast,z_pm,breakout_score,team_fga,opp_team_fga,team_pts,opp_team_pts
0,305,Robert Parish,Warriors,1979,1,1979-12-31,West,West,3,101.00,...,0.000000,0.000000,-0.254000,-1.655675,0.000000,-0.623703,159.000000,160.000000,223.000000,213.000000
1,76003,Kareem Abdul-Jabbar,Lakers,1979,1,1979-12-31,West,West,2,82.00,...,0.000000,0.000000,-1.986377,-0.750000,0.000000,-1.218188,271.991631,271.772985,336.166503,336.148597
2,76005,Tom Abernethy,Warriors,1979,1,1979-12-31,West,West,1,11.00,...,0.000000,0.000000,-1.667816,-1.771759,0.000000,-1.365436,159.000000,160.000000,223.000000,213.000000
3,76011,Alvan Adams,Suns,1979,1,1979-12-31,West,West,2,64.00,...,0.000000,0.000000,-0.634794,-0.862489,0.000000,-0.576144,181.000000,185.000000,219.000000,229.000000
4,76085,James Bailey,SuperSonics,1979,1,1979-12-31,West,West,4,41.00,...,0.000000,0.000000,-0.444750,2.886751,0.000000,0.643651,167.000000,170.000000,224.000000,217.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235264,1642942,Jahmai Mashack,Grizzlies,2025,51,2025-12-15,West,West,2,9.48,...,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,273.000000,277.000000,359.000000,343.000000
235265,1642948,Ryan Nembhard,Mavericks,2025,51,2025-12-15,West,West,3,85.60,...,-3.333333,11.145502,-0.222437,0.454344,-0.239259,-0.022767,270.000000,321.000000,363.000000,375.000000
235266,1642949,Yanic Konan Niederhauser,Clippers,2025,51,2025-12-15,West,West,1,10.20,...,-12.000000,28.586710,-0.611111,-0.750000,0.944495,-0.341657,236.000000,275.000000,307.000000,331.000000
235267,1642954,Will Richard,Warriors,2025,51,2025-12-15,West,West,1,19.15,...,6.000000,25.252063,-0.487263,-2.013996,-0.198004,-0.887431,181.000000,176.000000,217.000000,215.000000


In [6]:
features_overall_weekly_df.columns

Index(['player_id', 'full_name', 'team', 'season', 'week', 'week_start',
       'conference', 'pow_conference', 'games_played_this_week', 'numMinutes',
       'points', 'assists', 'blocks', 'steals', 'reboundsTotal',
       'reboundsDefensive', 'reboundsOffensive', 'fieldGoalsAttempted',
       'fieldGoalsMade', 'threePointersAttempted', 'threePointersMade',
       'freeThrowsAttempted', 'freeThrowsMade', 'turnovers', 'foulsPersonal',
       'plusMinusPoints', 'wins_this_week', 'wins_vs_team_with_all_nba_player',
       'is_win_vs_over_500', 'opponent_has_all_nba', 'avg_opp_score',
       'avg_opp_winrate_prior', 'avg_opp_wins_prior', 'avg_opp_losses_prior',
       'away_games_prior', 'away_losses_prior', 'away_win_streak_prior',
       'away_wins_prior', 'home_games_prior', 'home_losses_prior',
       'home_win_streak_prior', 'home_wins_prior', 'losses_prior',
       'wins_vs_over_500_prior', 'won_player_of_the_week',
       'all_star_this_season', 'mvp_this_season',
       'all_nba_f

In [7]:
query = """
WITH CTE AS (
SELECT team, season, week, SUM(assists) AS team_assists
FROM features_overall_weekly_df
GROUP BY team, season, week
)
,CTE2 AS (

SELECT team, season, week, SUM(fieldGoalsMade) AS team_fg
FROM features_overall_weekly_df
GROUP BY team, season, week
)
, CTE3 AS (
SELECT CTE.team, CTE.season, CTE.week, team_assists, team_fg
FROM CTE
JOIN CTE2
ON CTE.team = CTE2.team AND CTE.season = CTE2.season AND CTE.week = CTE2.week
)
, CTE4 AS (

SELECT team, season, week, SUM(points)/team_gms AS team_ppg
FROM features_overall_weekly_df
GROUP BY team, season, week, team_gms
)

, CTE5 AS (
SELECT *
,team_pts/team_gms AS team_ppg
FROM features_overall_weekly_df a
JOIN CTE3 b
ON a.team=b.team AND a.season = b.season AND a.week=b.week
)

, CTE6 AS (
  SELECT *,
  CASE WHEN team_ppg > 186 THEN 115 ELSE team_ppg END AS team_ppg_fixed  -- If team_ppg shows as greater than 186, set it to 115
  FROM CTE5
)

SELECT *,
AVG(team_ppg_fixed) OVER(PARTITION BY season, week) AS lg_ppg
FROM CTE6

"""
features_overall_weekly_df = duckdb.query(query).df().drop(columns=['team_1','season_1','week_1','team_ppg'])
features_overall_weekly_df

# League-wide statistics (scalars)
lg_AST = features_overall_weekly_df['assists'].mean()
lg_FG = features_overall_weekly_df['fieldGoalsMade'].mean()
lg_FT = features_overall_weekly_df['freeThrowsMade'].mean()
lg_PF = features_overall_weekly_df['foulsPersonal'].mean()
lg_FTA = features_overall_weekly_df['freeThrowsAttempted'].mean()

factor = (2/3) - (0.5 * (lg_AST / lg_FG)) / (2 * (lg_FG / lg_FT))
vop = features_overall_weekly_df['points'].mean() / (features_overall_weekly_df['fieldGoalsAttempted'].mean() - features_overall_weekly_df['reboundsOffensive'].mean() + features_overall_weekly_df['turnovers'].mean() + 0.44 * lg_FTA)
drb_pct = (features_overall_weekly_df['reboundsTotal'].mean() - features_overall_weekly_df['reboundsOffensive'].mean()) / features_overall_weekly_df['reboundsTotal'].mean()

features_overall_weekly_df['u_per'] = (1 / features_overall_weekly_df['numMinutes']) * (
    features_overall_weekly_df['threePointersMade']
    + (2/3) * features_overall_weekly_df['assists']
    + (2 - factor * (features_overall_weekly_df['team_assists'] / features_overall_weekly_df['team_fg'])) * features_overall_weekly_df['fieldGoalsMade']
    + (features_overall_weekly_df['freeThrowsMade'] * 0.5 * (1 + (1 - (features_overall_weekly_df['team_assists'] / features_overall_weekly_df['team_fg'])) + (2/3) * (features_overall_weekly_df['team_assists'] / features_overall_weekly_df['team_fg'])))
    - vop * features_overall_weekly_df['turnovers']
    - vop * drb_pct * (features_overall_weekly_df['fieldGoalsAttempted'] - features_overall_weekly_df['fieldGoalsMade'])
    - vop * 0.44 * (0.44 + (0.56 * drb_pct)) * (features_overall_weekly_df['freeThrowsAttempted'] - features_overall_weekly_df['freeThrowsMade'])
    + vop * (1 - drb_pct) * (features_overall_weekly_df['reboundsTotal'] - features_overall_weekly_df['reboundsOffensive'])
    + vop * drb_pct * features_overall_weekly_df['reboundsOffensive']
    + vop * features_overall_weekly_df['steals']
    + vop * drb_pct * features_overall_weekly_df['blocks']
    - features_overall_weekly_df['foulsPersonal'] * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * vop)
)

# First, check for problematic values
print("Minutes == 0:", (features_overall_weekly_df['numMinutes'] == 0).sum())
print("team_fg == 0:", (features_overall_weekly_df['team_fg'] == 0).sum())
print("NaN values in key columns:")
print(features_overall_weekly_df[['numMinutes', 'team_assists', 'team_fg']].isna().sum())

# Filter to only calculate for valid rows
valid_mask = (
    (features_overall_weekly_df['numMinutes'] > 0) &
    (features_overall_weekly_df['team_fg'] > 0) &
    features_overall_weekly_df['numMinutes'].notna()
)

# Initialize u_per with NaN
features_overall_weekly_df['u_per'] = np.nan

# Calculate only for valid rows
lg_AST = features_overall_weekly_df['assists'].mean()
lg_FG = features_overall_weekly_df['fieldGoalsMade'].mean()
lg_FT = features_overall_weekly_df['freeThrowsMade'].mean()
lg_PF = features_overall_weekly_df['foulsPersonal'].mean()
lg_FTA = features_overall_weekly_df['freeThrowsAttempted'].mean()

factor = (2/3) - (0.5 * (lg_AST / lg_FG)) / (2 * (lg_FG / lg_FT))
vop = features_overall_weekly_df['points'].mean() / (features_overall_weekly_df['fieldGoalsAttempted'].mean() - features_overall_weekly_df['reboundsOffensive'].mean() + features_overall_weekly_df['turnovers'].mean() + 0.44 * lg_FTA)
drb_pct = (features_overall_weekly_df['reboundsTotal'].mean() - features_overall_weekly_df['reboundsOffensive'].mean()) / features_overall_weekly_df['reboundsTotal'].mean()

features_overall_weekly_df.loc[valid_mask, 'u_per'] = (1 / features_overall_weekly_df.loc[valid_mask, 'numMinutes']) * (
    features_overall_weekly_df.loc[valid_mask, 'threePointersMade']
    + (2/3) * features_overall_weekly_df.loc[valid_mask, 'assists']
    + (2 - factor * (features_overall_weekly_df.loc[valid_mask, 'team_assists'] / features_overall_weekly_df.loc[valid_mask, 'team_fg'])) * features_overall_weekly_df.loc[valid_mask, 'fieldGoalsMade']
    + (features_overall_weekly_df.loc[valid_mask, 'freeThrowsMade'] * 0.5 * (1 + (1 - (features_overall_weekly_df.loc[valid_mask, 'team_assists'] / features_overall_weekly_df.loc[valid_mask, 'team_fg'])) + (2/3) * (features_overall_weekly_df.loc[valid_mask, 'team_assists'] / features_overall_weekly_df.loc[valid_mask, 'team_fg'])))
    - vop * features_overall_weekly_df.loc[valid_mask, 'turnovers']
    - vop * drb_pct * (features_overall_weekly_df.loc[valid_mask, 'fieldGoalsAttempted'] - features_overall_weekly_df.loc[valid_mask, 'fieldGoalsMade'])
    - vop * 0.44 * (0.44 + (0.56 * drb_pct)) * (features_overall_weekly_df.loc[valid_mask, 'freeThrowsAttempted'] - features_overall_weekly_df.loc[valid_mask, 'freeThrowsMade'])
    + vop * (1 - drb_pct) * (features_overall_weekly_df.loc[valid_mask, 'reboundsTotal'] - features_overall_weekly_df.loc[valid_mask, 'reboundsOffensive'])
    + vop * drb_pct * features_overall_weekly_df.loc[valid_mask, 'reboundsOffensive']
    + vop * features_overall_weekly_df.loc[valid_mask, 'steals']
    + vop * drb_pct * features_overall_weekly_df.loc[valid_mask, 'blocks']
    - features_overall_weekly_df.loc[valid_mask, 'foulsPersonal'] * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * vop)
)

# # Check results
# print("\nAfter filtering:")
# print("inf values:", np.isinf(features_overall_weekly_df['u_per']).sum())
# print("NaN values:", features_overall_weekly_df['u_per'].isna().sum())
# print("Valid u_per values:", valid_mask.sum())

features_overall_weekly_df['pace_adjustment'] = 2 * features_overall_weekly_df['lg_ppg'] / (features_overall_weekly_df['team_ppg_fixed'] + features_overall_weekly_df['avg_opp_score'])
features_overall_weekly_df['a_per'] = features_overall_weekly_df['pace_adjustment'] * features_overall_weekly_df['u_per']
league_avg_aPER = features_overall_weekly_df['a_per'].mean()
features_overall_weekly_df['player_efficiency_rating'] = features_overall_weekly_df['a_per'] * (15 / league_avg_aPER)

# Check final results
print("Final PER distribution:")
print(features_overall_weekly_df['player_efficiency_rating'].describe())
print()
print(f"League average PER: {features_overall_weekly_df['player_efficiency_rating'].mean():.2f} (should be ~15.0)")

# Apply minimum minutes threshold (common in PER calculations)
min_minutes = 5  # or whatever threshold makes sense for your data

features_overall_weekly_df.loc[
    features_overall_weekly_df['numMinutes'] < min_minutes,
    'player_efficiency_rating'
] = np.nan

print(f"\nAfter {min_minutes} minute filter:")
print(features_overall_weekly_df['player_efficiency_rating'].describe())

features_overall_weekly_df = features_overall_weekly_df#.drop(columns=['team_assists', 'team_fg','team_ppg_fixed', 'lg_ppg', 'u_per', 'pace_adjustment', 'a_per'])


Minutes == 0: 1250
team_fg == 0: 1
NaN values in key columns:
numMinutes      0
team_assists    0
team_fg         0
dtype: int64
Final PER distribution:
count    234018.000000
mean         15.000000
std          13.489267
min        -238.044097
25%           9.049632
50%          15.042475
75%          21.085482
max         715.447930
Name: player_efficiency_rating, dtype: float64

League average PER: 15.00 (should be ~15.0)

After 5 minute filter:
count    225026.000000
mean         15.208251
std          10.351617
min         -87.199716
25%           9.456756
50%          15.182895
75%          21.034017
max         162.659096
Name: player_efficiency_rating, dtype: float64


In [8]:
features_overall_weekly_df.columns

Index(['player_id', 'full_name', 'team', 'season', 'week', 'week_start',
       'conference', 'pow_conference', 'games_played_this_week', 'numMinutes',
       'points', 'assists', 'blocks', 'steals', 'reboundsTotal',
       'reboundsDefensive', 'reboundsOffensive', 'fieldGoalsAttempted',
       'fieldGoalsMade', 'threePointersAttempted', 'threePointersMade',
       'freeThrowsAttempted', 'freeThrowsMade', 'turnovers', 'foulsPersonal',
       'plusMinusPoints', 'wins_this_week', 'wins_vs_team_with_all_nba_player',
       'is_win_vs_over_500', 'opponent_has_all_nba', 'avg_opp_score',
       'avg_opp_winrate_prior', 'avg_opp_wins_prior', 'avg_opp_losses_prior',
       'away_games_prior', 'away_losses_prior', 'away_win_streak_prior',
       'away_wins_prior', 'home_games_prior', 'home_losses_prior',
       'home_win_streak_prior', 'home_wins_prior', 'losses_prior',
       'wins_vs_over_500_prior', 'won_player_of_the_week',
       'all_star_this_season', 'mvp_this_season',
       'all_nba_f

In [9]:
features_overall_weekly_df = features_overall_weekly_df.rename(columns={'team_ppg_fixed':'team_ppg'}).drop(columns=['u_per', 'pace_adjustment', 'a_per'])


In [10]:
# ============================================================================
# COMPREHENSIVE DATA QUALITY CHECK ON BASE DATASET
# ============================================================================

print("="*70)
print("BASE DATASET QUALITY CHECKS")
print("="*70)

# Start fresh with original data
df = features_overall_weekly_df.copy()

print(f"\nTotal rows: {len(df)}")

# ============================================================================
# 1. CHECK FOR DUPLICATES
# ============================================================================
print("\n1. DUPLICATES:")
duplicates = df.duplicated(subset=['player_id', 'season', 'week', 'week_start'], keep=False)
print(f"   Duplicate player-week records: {duplicates.sum()}")

if duplicates.sum() > 0:
    print("   Sample duplicates:")
    sample_dups = df[duplicates].sort_values(['player_id', 'season', 'week']).head(10)
    print(sample_dups[['player_id', 'full_name', 'team', 'season', 'week', 'week_start', 'points', 'numMinutes']])

# ============================================================================
# 2. CHECK MINUTES PER GAME
# ============================================================================
print("\n2. MINUTES PER GAME:")
df['mpg_check'] = df['numMinutes'] / df['games_played_this_week']
print(f"   Max MPG: {df['mpg_check'].max():.1f}")
print(f"   Rows with MPG > 48: {(df['mpg_check'] > 48).sum()}")
print(f"   Rows with MPG > 60: {(df['mpg_check'] > 60).sum()}")

if (df['mpg_check'] > 48).sum() > 0:
    print("\n   Sample high MPG:")
    print(df[df['mpg_check'] > 48].nlargest(5, 'mpg_check')[[
        'full_name', 'team', 'season', 'week', 'week_start', 'games_played_this_week', 'numMinutes', 'mpg_check'
    ]])

# ============================================================================
# 3. CHECK TEAM STATS CONSISTENCY
# ============================================================================
print("\n3. TEAM STATS:")

# Check if team_ppg matches calculated value
df['team_ppg_calc'] = df['team_pts'] / df['games_played_this_week']
ppg_mismatch = (abs(df['team_ppg'] - df['team_ppg_calc']) > 1).fillna(False)

print(f"   Rows where team_ppg != team_pts/games: {ppg_mismatch.sum()}")
print(f"   team_ppg > 150: {(df['team_ppg'] > 150).sum()}")
print(f"   team_ppg < 40: {(df['team_ppg'] < 40).sum()}")

if (df['team_ppg'] > 150).sum() > 0:
    print("\n   Sample extreme team_ppg:")
    print(df[df['team_ppg'] > 150][['full_name', 'team', 'season', 'week', 'week_start',
                                     'games_played_this_week', 'team_pts', 'team_ppg', 'team_ppg_calc']].head(5))

# ============================================================================
# 4. CHECK GAMES PLAYED
# ============================================================================
print("\n4. GAMES PLAYED:")
print(f"   Min games: {df['games_played_this_week'].min()}")
print(f"   Max games: {df['games_played_this_week'].max()}")
print(f"   Games > 5: {(df['games_played_this_week'] > 5).sum()}")
print(f"   Games = 0: {(df['games_played_this_week'] == 0).sum()}")

# ============================================================================
# 5. CHECK WINS
# ============================================================================
print("\n5. WINS:")
print(f"   Max wins_this_week: {df['wins_this_week'].max()}")
print(f"   Wins > games: {(df['wins_this_week'] > df['games_played_this_week']).sum()}")

if (df['wins_this_week'] > df['games_played_this_week']).sum() > 0:
    print("\n   Sample wins > games:")
    print(df[df['wins_this_week'] > df['games_played_this_week']][
        ['full_name', 'team', 'season', 'week', 'week_start', 'games_played_this_week', 'wins_this_week']
    ].head(5))

# ============================================================================
# 6. CHECK OPPONENT STATS
# ============================================================================
print("\n6. OPPONENT STATS:")
df['opp_ppg_calc'] = df['opp_team_pts'] / df['games_played_this_week']
print(f"   Max opp_team_pts: {df['opp_team_pts'].max()}")
print(f"   opp_ppg > 150: {(df['opp_ppg_calc'] > 150).sum()}")

# ============================================================================
# 7. SUMMARY OF ISSUES
# ============================================================================
print("\n" + "="*70)
print("SUMMARY OF DATA QUALITY ISSUES")
print("="*70)

issues_mask = (
    duplicates |
    (df['mpg_check'] > 48) |
    ppg_mismatch |
    (df['team_ppg'] > 150) |
    (df['team_ppg'] < 40) |
    (df['games_played_this_week'] > 5) |
    (df['games_played_this_week'] == 0) |
    (df['wins_this_week'] > df['games_played_this_week'])
)

print(f"Total rows with issues: {issues_mask.sum()} ({issues_mask.sum()/len(df)*100:.1f}%)")
print(f"Clean rows: {(~issues_mask).sum()} ({(~issues_mask).sum()/len(df)*100:.1f}%)")

# Show breakdown
print("\nBreakdown by issue type:")
print(f"  - Duplicates: {duplicates.sum()}")
print(f"  - MPG > 48: {(df['mpg_check'] > 48).sum()}")
print(f"  - team_ppg mismatch: {ppg_mismatch.sum()}")
print(f"  - team_ppg > 150: {(df['team_ppg'] > 150).sum()}")
print(f"  - team_ppg < 40: {(df['team_ppg'] < 40).sum()}")
print(f"  - games > 5: {(df['games_played_this_week'] > 5).sum()}")
print(f"  - games = 0: {(df['games_played_this_week'] == 0).sum()}")
print(f"  - wins > games: {(df['wins_this_week'] > df['games_played_this_week']).sum()}")

print("\n" + "="*70)

BASE DATASET QUALITY CHECKS

Total rows: 235269

1. DUPLICATES:
   Duplicate player-week records: 11411
   Sample duplicates:
        player_id       full_name         team  season  week week_start  \
121768         56     Gary Payton  SuperSonics    2002     8 2003-02-17   
121769         56     Gary Payton        Bucks    2002     8 2003-02-17   
104441         72  Kenny Anderson      Hornets    1995     9 1996-02-26   
104442         72  Kenny Anderson      Hornets    1995     9 1996-02-26   
121772         72  Kenny Anderson      Hornets    2002     8 2003-02-17   
121773         72  Kenny Anderson      Hornets    2002     8 2003-02-17   
106543         72  Kenny Anderson      Hornets    2002     9 2003-02-24   
106544         72  Kenny Anderson      Hornets    2002     9 2003-02-24   
197843         72  Kenny Anderson      Hornets    2002    10 2003-03-03   
197844         72  Kenny Anderson      Hornets    2002    10 2003-03-03   

        points  numMinutes  
121768    31.0     

In [11]:
# ============================================================================
# CLEAN THE BASE DATASET
# ============================================================================

print("Cleaning base dataset...")
df_clean = features_overall_weekly_df.copy()

# ============================================================================
# FIX 1: Remove duplicates (keep first occurrence)
# ============================================================================
print(f"\n1. Removing duplicates...")
print(f"   Before: {len(df_clean)}")
df_clean = df_clean.drop_duplicates(
    subset=['player_id', 'season', 'week', 'week_start'],
    keep='first'
)
print(f"   After: {len(df_clean)}")

# ============================================================================
# FIX 2: Recalculate team_ppg and opp_ppg from raw stats
# ============================================================================
print(f"\n2. Recalculating team_ppg and opp_ppg...")
df_clean['team_ppg'] = df_clean['team_pts'] / df_clean['games_played_this_week']
df_clean['opp_ppg'] = df_clean['opp_team_pts'] / df_clean['games_played_this_week']

# ============================================================================
# FIX 3: Calculate MPG
# ============================================================================
df_clean['mpg'] = df_clean['numMinutes'] / df_clean['games_played_this_week']

# ============================================================================
# FIX 4: Filter out impossible data
# ============================================================================
print(f"\n3. Filtering impossible data...")
print(f"   Before: {len(df_clean)}")

valid_mask = (
    (df_clean['mpg'] > 0) &
    (df_clean['mpg'] <= 48) &  # No one can play more than 48 MPG
    (df_clean['team_ppg'] >= 40) &  # Reasonable team scoring
    (df_clean['team_ppg'] <= 150) &
    (df_clean['games_played_this_week'] > 0) &
    (df_clean['games_played_this_week'] <= 5)  # Max 5 games in a week
)

df_clean = df_clean[valid_mask].copy()
print(f"   After: {len(df_clean)}")

# ============================================================================
# FIX 5: Fix wins_this_week if it's cumulative
# ============================================================================
print(f"\n4. Checking wins_this_week...")
wins_issue = (df_clean['wins_this_week'] > df_clean['games_played_this_week']).sum()
print(f"   Rows where wins > games: {wins_issue}")

if wins_issue > 0:
    print("   NOTE: wins_this_week appears to contain season totals, not weekly wins")
    print("   Capping wins at games_played for Win Shares calculation")
    df_clean['wins_this_week_clean'] = df_clean[['wins_this_week', 'games_played_this_week']].min(axis=1)
else:
    df_clean['wins_this_week_clean'] = df_clean['wins_this_week']

# ============================================================================
# SUMMARY OF CLEAN DATA
# ============================================================================
print("\n" + "="*70)
print("CLEAN DATASET SUMMARY")
print("="*70)
print(f"Original rows: 235,269")
print(f"Clean rows: {len(df_clean):,}")
print(f"Removed: {235269 - len(df_clean):,} ({(235269 - len(df_clean))/235269*100:.1f}%)")

print("\nData quality checks on clean data:")
print(f"  Max MPG: {df_clean['mpg'].max():.1f}")
print(f"  Mean MPG: {df_clean['mpg'].mean():.1f}")
print(f"  Max team_ppg: {df_clean['team_ppg'].max():.1f}")
print(f"  Min team_ppg: {df_clean['team_ppg'].min():.1f}")
print(f"  Max games/week: {df_clean['games_played_this_week'].max()}")
print(f"  Wins > games: {(df_clean['wins_this_week_clean'] > df_clean['games_played_this_week']).sum()}")

# Update the main dataframe
features_overall_weekly_df = df_clean

print("\n" + "="*70)
print("Dataset cleaned and ready for Win Shares calculation!")
print("="*70)

Cleaning base dataset...

1. Removing duplicates...
   Before: 235269
   After: 229553

2. Recalculating team_ppg and opp_ppg...

3. Filtering impossible data...
   Before: 229553
   After: 159437

4. Checking wins_this_week...
   Rows where wins > games: 6141
   NOTE: wins_this_week appears to contain season totals, not weekly wins
   Capping wins at games_played for Win Shares calculation

CLEAN DATASET SUMMARY
Original rows: 235,269
Clean rows: 159,437
Removed: 75,832 (32.2%)

Data quality checks on clean data:
  Max MPG: 48.0
  Mean MPG: 24.9
  Max team_ppg: 150.0
  Min team_ppg: 40.0
  Max games/week: 5
  Wins > games: 0

Dataset cleaned and ready for Win Shares calculation!


In [12]:
# ============================================================================
# WIN SHARES CALCULATION ON CLEAN DATA
# ============================================================================

import numpy as np

print("Calculating Win Shares on clean dataset...")

# Calculate ppg first
features_overall_weekly_df['ppg'] = (
    features_overall_weekly_df['points'] / features_overall_weekly_df['games_played_this_week']
)

# ============================================================================
# STEP 1: Calculate Possessions
# ============================================================================

# Player possessions (weekly total)
features_overall_weekly_df['player_poss'] = (
    features_overall_weekly_df['fieldGoalsAttempted'] +
    0.44 * features_overall_weekly_df['freeThrowsAttempted'] +
    features_overall_weekly_df['turnovers'] -
    features_overall_weekly_df['reboundsOffensive']
)

# Team possessions (weekly total)
features_overall_weekly_df['team_poss'] = (
    features_overall_weekly_df['team_fga'] -
    features_overall_weekly_df['reboundsOffensive'] +
    features_overall_weekly_df['turnovers'] +
    0.44 * features_overall_weekly_df['freeThrowsAttempted']
)

# Opponent possessions (weekly total)
features_overall_weekly_df['opp_poss'] = (
    features_overall_weekly_df['opp_team_fga'] +
    features_overall_weekly_df['turnovers'] +
    0.44 * features_overall_weekly_df['freeThrowsAttempted']
)

# ============================================================================
# STEP 2: League Averages
# ============================================================================

lg_pts = features_overall_weekly_df['points'].sum()
lg_poss = features_overall_weekly_df['player_poss'].sum()
lg_ortg = (lg_pts / lg_poss) * 100

lg_opp_pts = features_overall_weekly_df['opp_team_pts'].sum()
lg_opp_poss = features_overall_weekly_df['opp_poss'].sum()
lg_drtg = (lg_opp_pts / lg_opp_poss) * 100

print(f"\nLeague Offensive Rating: {lg_ortg:.2f}")
print(f"League Defensive Rating: {lg_drtg:.2f}")

# ============================================================================
# STEP 3: Offensive Win Shares
# ============================================================================

# Player offensive rating
features_overall_weekly_df['ortg'] = np.where(
    features_overall_weekly_df['player_poss'] > 0,
    (features_overall_weekly_df['points'] / features_overall_weekly_df['player_poss']) * 100,
    lg_ortg
)

# Marginal offense (points above/below league average)
features_overall_weekly_df['marginal_offense'] = (
    features_overall_weekly_df['player_poss'] *
    (features_overall_weekly_df['ortg'] - lg_ortg) / 100
)

# Convert to win shares (marginal points per win ~30 for NBA)
marginal_ppw = 30
features_overall_weekly_df['ows'] = features_overall_weekly_df['marginal_offense'] / marginal_ppw

# ============================================================================
# STEP 4: Defensive Win Shares
# ============================================================================

# Team defensive rating
features_overall_weekly_df['team_drtg'] = np.where(
    features_overall_weekly_df['opp_poss'] > 0,
    (features_overall_weekly_df['opp_team_pts'] / features_overall_weekly_df['opp_poss']) * 100,
    lg_drtg
)

# Calculate team minutes for the week
team_weekly_min = features_overall_weekly_df.groupby(
    ['team', 'season', 'week', 'week_start']
)['numMinutes'].sum().reset_index()
team_weekly_min.columns = ['team', 'season', 'week', 'week_start', 'team_min_week']

features_overall_weekly_df = features_overall_weekly_df.merge(
    team_weekly_min,
    on=['team', 'season', 'week', 'week_start'],
    how='left'
)

# Player's share of team minutes
features_overall_weekly_df['min_pct'] = (
    features_overall_weekly_df['numMinutes'] / features_overall_weekly_df['team_min_week']
)

# Defensive contribution (stops)
features_overall_weekly_df['stops'] = (
    features_overall_weekly_df['steals'] +
    features_overall_weekly_df['blocks'] * 0.5 +
    features_overall_weekly_df['reboundsDefensive'] * 0.5 -
    features_overall_weekly_df['foulsPersonal'] * 0.4
)

# Stop rate per 48 minutes
features_overall_weekly_df['stop_rate'] = np.where(
    features_overall_weekly_df['numMinutes'] > 0,
    (features_overall_weekly_df['stops'] / features_overall_weekly_df['numMinutes']) * 48,
    0
)

lg_stop_rate = (features_overall_weekly_df['stops'].sum() /
                features_overall_weekly_df['numMinutes'].sum()) * 48

# Player defensive rating (adjusted from team based on stops)
features_overall_weekly_df['drtg'] = (
    features_overall_weekly_df['team_drtg'] -
    (features_overall_weekly_df['stop_rate'] - lg_stop_rate) * 0.2
)

# Marginal defense (points saved vs league average)
features_overall_weekly_df['marginal_defense'] = (
    features_overall_weekly_df['min_pct'] *
    features_overall_weekly_df['opp_poss'] *
    (lg_drtg - features_overall_weekly_df['drtg']) / 100
)

features_overall_weekly_df['dws'] = features_overall_weekly_df['marginal_defense'] / marginal_ppw

# ============================================================================
# STEP 5: Total Win Shares (before scaling)
# ============================================================================

features_overall_weekly_df['ws_unscaled'] = (
    features_overall_weekly_df['ows'] + features_overall_weekly_df['dws']
)

# ============================================================================
# STEP 6: Scale to Team Wins
# ============================================================================

# Sum marginal wins by team-week
team_marginal = features_overall_weekly_df.groupby(
    ['team', 'season', 'week', 'week_start']
)['ws_unscaled'].sum().reset_index()
team_marginal.columns = ['team', 'season', 'week', 'week_start', 'team_marginal_wins']

# Get team wins
team_wins = features_overall_weekly_df.groupby(
    ['team', 'season', 'week', 'week_start']
)['wins_this_week_clean'].first().reset_index()

# Merge
scaling_data = team_marginal.merge(team_wins, on=['team', 'season', 'week', 'week_start'])

# Calculate scaling factor (avoid division by zero)
scaling_data['scale_factor'] = np.where(
    scaling_data['team_marginal_wins'].abs() > 0.01,
    scaling_data['wins_this_week_clean'] / scaling_data['team_marginal_wins'],
    1.0
)

# Merge back
features_overall_weekly_df = features_overall_weekly_df.merge(
    scaling_data[['team', 'season', 'week', 'week_start', 'scale_factor']],
    on=['team', 'season', 'week', 'week_start'],
    how='left'
)

# Apply scaling
features_overall_weekly_df['win_shares'] = (
    features_overall_weekly_df['ws_unscaled'] * features_overall_weekly_df['scale_factor']
)
features_overall_weekly_df['offensive_win_shares'] = (
    features_overall_weekly_df['ows'] * features_overall_weekly_df['scale_factor']
)
features_overall_weekly_df['defensive_win_shares'] = (
    features_overall_weekly_df['dws'] * features_overall_weekly_df['scale_factor']
)

# ============================================================================
# RESULTS
# ============================================================================

analysis = features_overall_weekly_df[features_overall_weekly_df['mpg'] >= 5].copy()

print("\n" + "="*70)
print("WIN SHARES RESULTS (CLEAN DATA)")
print("="*70)

print("\nWin Shares Distribution (5+ MPG):")
print(analysis['win_shares'].describe())
print()

print("Offensive Win Shares:")
print(analysis['offensive_win_shares'].describe())
print()

print("Defensive Win Shares:")
print(analysis['defensive_win_shares'].describe())
print()

print("Top 10 by Win Shares:")
top10 = analysis.nlargest(10, 'win_shares')[[
    'full_name', 'team', 'season', 'week', 'week_start',
    'wins_this_week_clean', 'games_played_this_week',
    'mpg', 'ppg', 'offensive_win_shares', 'defensive_win_shares', 'win_shares'
]]
print(top10.to_string(index=False))

print("\nBottom 10:")
bottom10 = analysis.nsmallest(10, 'win_shares')[[
    'full_name', 'team', 'season', 'week', 'week_start',
    'wins_this_week_clean', 'games_played_this_week',
    'mpg', 'ppg', 'offensive_win_shares', 'defensive_win_shares', 'win_shares'
]]
print(bottom10.to_string(index=False))

print("\n" + "="*70)
print("VERIFICATION")
print("="*70)

verification = analysis.groupby(['team', 'season', 'week', 'week_start']).agg({
    'wins_this_week_clean': 'first',
    'games_played_this_week': 'first',
    'win_shares': 'sum'
}).reset_index()

verification['difference'] = abs(verification['wins_this_week_clean'] - verification['win_shares'])

print(f"Team-weeks analyzed: {len(verification)}")
print(f"Perfect matches (diff < 0.001): {(verification['difference'] < 0.001).sum()}")
print(f"Close matches (diff < 0.1): {(verification['difference'] < 0.1).sum()}")
print(f"Average difference: {verification['difference'].mean():.6f}")
print(f"Max difference: {verification['difference'].max():.6f}")

print("\n" + "="*70)
print(f"Final analysis set: {len(analysis):,} player-weeks (5+ MPG)")
print(f"Mean Win Shares: {analysis['win_shares'].mean():.4f}")
print(f"Median Win Shares: {analysis['win_shares'].median():.4f}")
print(f"Std Dev: {analysis['win_shares'].std():.4f}")
print("="*70)

Calculating Win Shares on clean dataset...

League Offensive Rating: 105.98
League Defensive Rating: 119.55

WIN SHARES RESULTS (CLEAN DATA)

Win Shares Distribution (5+ MPG):
count    155588.000000
mean          0.226200
std           4.649639
min        -186.643303
25%          -0.115512
50%           0.064939
75%           0.572832
max         305.745206
Name: win_shares, dtype: float64

Offensive Win Shares:
count    155588.000000
mean          0.092423
std           4.963322
min        -269.361523
25%          -0.233215
50%           0.000000
75%           0.425496
max         351.095243
Name: offensive_win_shares, dtype: float64

Defensive Win Shares:
count    155588.000000
mean          0.133776
std           1.993489
min        -295.145785
25%           0.000000
50%           0.056140
75%           0.229127
max         130.729162
Name: defensive_win_shares, dtype: float64

Top 10 by Win Shares:
         full_name         team  season  week week_start  wins_this_week_clean  game

In [13]:
# ============================================================================
# DIAGNOSE SCALING ISSUES
# ============================================================================

print("Diagnosing scaling issues...")

# Check the scaling factors
scaling_check = features_overall_weekly_df.groupby(['team', 'season', 'week', 'week_start']).agg({
    'wins_this_week_clean': 'first',
    'ws_unscaled': 'sum',
    'scale_factor': 'first'
}).reset_index()

scaling_check['team_ws_unscaled'] = scaling_check['ws_unscaled']

print("\nScaling factor distribution:")
print(scaling_check['scale_factor'].describe())

print("\nExtreme scaling factors:")
extreme = scaling_check[(scaling_check['scale_factor'] > 10) | (scaling_check['scale_factor'] < -10)]
print(f"Team-weeks with scale_factor > 10 or < -10: {len(extreme)}")

if len(extreme) > 0:
    print("\nSample extreme cases:")
    print(extreme.nlargest(10, 'scale_factor')[
        ['team', 'season', 'week', 'week_start', 'wins_this_week_clean', 'team_ws_unscaled', 'scale_factor']
    ])

# Check for near-zero unscaled win shares
near_zero = scaling_check[scaling_check['team_ws_unscaled'].abs() < 0.1]
print(f"\nTeam-weeks with near-zero unscaled WS: {len(near_zero)}")

if len(near_zero) > 0:
    print("\nSample near-zero cases:")
    print(near_zero.head(10)[
        ['team', 'season', 'week', 'week_start', 'wins_this_week_clean', 'team_ws_unscaled', 'scale_factor']
    ])

Diagnosing scaling issues...

Scaling factor distribution:
count    19678.000000
mean         0.600385
std         16.738006
min       -344.498514
25%         -1.382865
50%          0.803825
75%          2.807209
max        374.818116
Name: scale_factor, dtype: float64

Extreme scaling factors:
Team-weeks with scale_factor > 10 or < -10: 2303

Sample extreme cases:
            team  season  week week_start  wins_this_week_clean  \
11608    Nuggets    1987    15 1988-04-11                   4.0   
12869     Pacers    2024    12 2025-03-17                   4.0   
4713   Grizzlies    2018    46 2018-11-12                   4.0   
1264       Bucks    2011    11 2012-03-12                   3.0   
1580     Bullets    1984     4 1985-01-21                   4.0   
8232      Knicks    1980    50 1980-12-08                   3.0   
6679     Hornets    2018    14 2019-04-01                   4.0   
8162       Kings    2022    44 2022-10-31                   3.0   
10609  Mavericks    2012    1

In [14]:
# ============================================================================
# FIXED WIN SHARES - ROBUST SCALING
# ============================================================================

print("Recalculating Win Shares with robust scaling...")

# Instead of scaling, let's use a different approach:
# Distribute team wins proportionally to player contributions

# For each team-week, calculate each player's share of team contributions
team_totals = features_overall_weekly_df.groupby(['team', 'season', 'week', 'week_start']).agg({
    'ows': 'sum',
    'dws': 'sum',
    'wins_this_week_clean': 'first'
}).reset_index()
team_totals.columns = ['team', 'season', 'week', 'week_start', 'team_ows_sum', 'team_dws_sum', 'team_wins']

# Calculate total offensive and defensive contributions
team_totals['team_off_contrib'] = team_totals['team_ows_sum']
team_totals['team_def_contrib'] = team_totals['team_dws_sum']
team_totals['team_total_contrib'] = team_totals['team_off_contrib'] + team_totals['team_def_contrib']

# Merge back
features_overall_weekly_df = features_overall_weekly_df.merge(
    team_totals[['team', 'season', 'week', 'week_start', 'team_off_contrib', 'team_def_contrib', 'team_total_contrib', 'team_wins']],
    on=['team', 'season', 'week', 'week_start'],
    how='left',
    suffixes=('', '_new')
)

# Calculate player's share of team wins based on their contribution
# Use absolute values to handle negative contributions properly
features_overall_weekly_df['abs_contrib'] = (
    features_overall_weekly_df['ows'].abs() + features_overall_weekly_df['dws'].abs()
)

team_abs_contrib = features_overall_weekly_df.groupby(['team', 'season', 'week', 'week_start'])['abs_contrib'].sum().reset_index()
team_abs_contrib.columns = ['team', 'season', 'week', 'week_start', 'team_abs_contrib']

features_overall_weekly_df = features_overall_weekly_df.merge(
    team_abs_contrib,
    on=['team', 'season', 'week', 'week_start'],
    how='left'
)

# Player's win share = their proportion of total contribution * team wins
# But preserve the sign (positive/negative) from original WS
features_overall_weekly_df['contrib_share'] = np.where(
    features_overall_weekly_df['team_abs_contrib'] > 0,
    features_overall_weekly_df['abs_contrib'] / features_overall_weekly_df['team_abs_contrib'],
    1.0 / features_overall_weekly_df.groupby(['team', 'season', 'week', 'week_start'])['player_id'].transform('count')
)

# Assign wins proportionally, preserving sign
features_overall_weekly_df['ws_magnitude'] = (
    features_overall_weekly_df['contrib_share'] * features_overall_weekly_df['team_wins']
)

# Apply sign based on whether player was net positive or negative
features_overall_weekly_df['ws_signed'] = (
    features_overall_weekly_df['ws_magnitude'] *
    np.sign(features_overall_weekly_df['ows'] + features_overall_weekly_df['dws'])
)

# For players with exactly zero contribution, give them zero WS
features_overall_weekly_df['win_shares'] = np.where(
    features_overall_weekly_df['abs_contrib'] > 0,
    features_overall_weekly_df['ws_signed'],
    0
)

# Split into offensive and defensive proportionally
features_overall_weekly_df['off_share'] = np.where(
    (features_overall_weekly_df['ows'].abs() + features_overall_weekly_df['dws'].abs()) > 0,
    features_overall_weekly_df['ows'].abs() / (features_overall_weekly_df['ows'].abs() + features_overall_weekly_df['dws'].abs()),
    0.5
)

features_overall_weekly_df['offensive_win_shares'] = (
    features_overall_weekly_df['win_shares'] * features_overall_weekly_df['off_share'] * np.sign(features_overall_weekly_df['ows'])
)

features_overall_weekly_df['defensive_win_shares'] = (
    features_overall_weekly_df['win_shares'] * (1 - features_overall_weekly_df['off_share']) * np.sign(features_overall_weekly_df['dws'])
)

# ============================================================================
# RESULTS
# ============================================================================

analysis = features_overall_weekly_df[features_overall_weekly_df['mpg'] >= 5].copy()

print("\n" + "="*70)
print("WIN SHARES RESULTS (ROBUST SCALING)")
print("="*70)

print("\nWin Shares Distribution (5+ MPG):")
print(analysis['win_shares'].describe())
print()

print("Offensive Win Shares:")
print(analysis['offensive_win_shares'].describe())
print()

print("Defensive Win Shares:")
print(analysis['defensive_win_shares'].describe())
print()

print("Top 10 by Win Shares:")
top10 = analysis.nlargest(10, 'win_shares')[[
    'full_name', 'team', 'season', 'week', 'week_start',
    'team_wins', 'games_played_this_week',
    'mpg', 'ppg', 'offensive_win_shares', 'defensive_win_shares', 'win_shares'
]]
print(top10.to_string(index=False))

print("\nBottom 10:")
bottom10 = analysis.nsmallest(10, 'win_shares')[[
    'full_name', 'team', 'season', 'week', 'week_start',
    'team_wins', 'games_played_this_week',
    'mpg', 'ppg', 'offensive_win_shares', 'defensive_win_shares', 'win_shares'
]]
print(bottom10.to_string(index=False))

print("\n" + "="*70)
print("VERIFICATION")
print("="*70)

verification = analysis.groupby(['team', 'season', 'week', 'week_start']).agg({
    'team_wins': 'first',
    'games_played_this_week': 'first',
    'win_shares': 'sum'
}).reset_index()

verification['difference'] = abs(verification['team_wins'] - verification['win_shares'])

print(f"Team-weeks analyzed: {len(verification)}")
print(f"Perfect matches (diff < 0.001): {(verification['difference'] < 0.001).sum()}")
print(f"Close matches (diff < 0.5): {(verification['difference'] < 0.5).sum()}")
print(f"Average difference: {verification['difference'].mean():.6f}")
print(f"Max difference: {verification['difference'].max():.6f}")

print("\n" + "="*70)
print(f"Final analysis set: {len(analysis):,} player-weeks (5+ MPG)")
print(f"Mean Win Shares: {analysis['win_shares'].mean():.4f}")
print(f"Median Win Shares: {analysis['win_shares'].median():.4f}")
print(f"Std Dev: {analysis['win_shares'].std():.4f}")
print("="*70)

Recalculating Win Shares with robust scaling...

WIN SHARES RESULTS (ROBUST SCALING)

Win Shares Distribution (5+ MPG):
count    155588.000000
mean          0.031058
std           0.363777
min          -5.000000
25%          -0.121172
50%           0.000000
75%           0.175267
max           4.000000
Name: win_shares, dtype: float64

Offensive Win Shares:
count    155588.000000
mean          0.146369
std           0.220669
min          -1.946655
25%           0.000721
50%           0.086796
75%           0.208645
max           3.853187
Name: offensive_win_shares, dtype: float64

Defensive Win Shares:
count    155588.000000
mean          0.027890
std           0.145799
min          -1.703729
25%          -0.015851
50%           0.002091
75%           0.050198
max           5.000000
Name: defensive_win_shares, dtype: float64

Top 10 by Win Shares:
        full_name      team  season  week week_start  team_wins  games_played_this_week   mpg   ppg  offensive_win_shares  defensive_win_sha

In [15]:
# ============================================================================
# FINAL WIN SHARES - SIMPLE PROPORTIONAL DISTRIBUTION
# ============================================================================

print("Calculating Win Shares - final approach...")

# Calculate unscaled WS (marginal contributions)
features_overall_weekly_df['ws_marginal'] = features_overall_weekly_df['ows'] + features_overall_weekly_df['dws']

# For each team-week, get total marginal WS and actual wins
team_summary = features_overall_weekly_df.groupby(['team', 'season', 'week', 'week_start']).agg({
    'ws_marginal': 'sum',
    'ows': 'sum',
    'dws': 'sum',
    'wins_this_week_clean': 'first'
}).reset_index()

team_summary.columns = ['team', 'season', 'week', 'week_start', 'team_marginal_ws', 'team_ows', 'team_dws', 'team_wins']

# Merge back
features_overall_weekly_df = features_overall_weekly_df.merge(
    team_summary,
    on=['team', 'season', 'week', 'week_start'],
    how='left',
    suffixes=('', '_drop')
)

# Calculate scaling factor with safeguard
features_overall_weekly_df['ws_scale'] = np.where(
    features_overall_weekly_df['team_marginal_ws'].abs() > 0.01,
    features_overall_weekly_df['team_wins'] / features_overall_weekly_df['team_marginal_ws'],
    0  # If team had no marginal contribution, everyone gets 0
)

# Cap extreme scaling factors
features_overall_weekly_df['ws_scale_capped'] = features_overall_weekly_df['ws_scale'].clip(-5, 5)

# Apply scaling
features_overall_weekly_df['win_shares'] = features_overall_weekly_df['ws_marginal'] * features_overall_weekly_df['ws_scale_capped']
features_overall_weekly_df['offensive_win_shares'] = features_overall_weekly_df['ows'] * features_overall_weekly_df['ws_scale_capped']
features_overall_weekly_df['defensive_win_shares'] = features_overall_weekly_df['dws'] * features_overall_weekly_df['ws_scale_capped']

# ============================================================================
# RESULTS
# ============================================================================

analysis = features_overall_weekly_df[features_overall_weekly_df['mpg'] >= 5].copy()

print("\n" + "="*70)
print("WIN SHARES - FINAL RESULTS")
print("="*70)

print("\nWin Shares Distribution (5+ MPG):")
print(analysis['win_shares'].describe())
print()

print("Offensive Win Shares:")
print(analysis['offensive_win_shares'].describe())
print()

print("Defensive Win Shares:")
print(analysis['defensive_win_shares'].describe())
print()

print("Top 10 by Win Shares:")
top10 = analysis.nlargest(10, 'win_shares')[[
    'full_name', 'team', 'season', 'week', 'week_start',
    'team_wins', 'games_played_this_week',
    'mpg', 'ppg', 'offensive_win_shares', 'defensive_win_shares', 'win_shares'
]]
print(top10.to_string(index=False))

print("\nBottom 10:")
bottom10 = analysis.nsmallest(10, 'win_shares')[[
    'full_name', 'team', 'season', 'week', 'week_start',
    'team_wins', 'games_played_this_week',
    'mpg', 'ppg', 'offensive_win_shares', 'defensive_win_shares', 'win_shares'
]]
print(bottom10.to_string(index=False))

print("\n" + "="*70)
print("VERIFICATION (Win Shares should sum to team wins)")
print("="*70)

verification = analysis.groupby(['team', 'season', 'week', 'week_start']).agg({
    'team_wins': 'first',
    'games_played_this_week': 'first',
    'win_shares': 'sum',
    'ws_scale_capped': 'first'
}).reset_index()

verification['difference'] = abs(verification['team_wins'] - verification['win_shares'])

print(f"\nTeam-weeks analyzed: {len(verification)}")
print(f"Perfect matches (diff < 0.001): {(verification['difference'] < 0.001).sum()} ({(verification['difference'] < 0.001).sum()/len(verification)*100:.1f}%)")
print(f"Close matches (diff < 0.1): {(verification['difference'] < 0.1).sum()} ({(verification['difference'] < 0.1).sum()/len(verification)*100:.1f}%)")
print(f"Close matches (diff < 0.5): {(verification['difference'] < 0.5).sum()} ({(verification['difference'] < 0.5).sum()/len(verification)*100:.1f}%)")
print(f"\nAverage difference: {verification['difference'].mean():.4f}")
print(f"Median difference: {verification['difference'].median():.4f}")
print(f"Max difference: {verification['difference'].max():.4f}")

# Show some examples where scaling was capped
capped = verification[verification['ws_scale_capped'].abs() == 5]
print(f"\nTeam-weeks where scaling was capped at ±5: {len(capped)}")

print("\n" + "="*70)
print(f"Final analysis set: {len(analysis):,} player-weeks (5+ MPG)")
print(f"Mean Win Shares: {analysis['win_shares'].mean():.4f}")
print(f"Median Win Shares: {analysis['win_shares'].median():.4f}")
print(f"Std Dev: {analysis['win_shares'].std():.4f}")
print("="*70)

Calculating Win Shares - final approach...

WIN SHARES - FINAL RESULTS

Win Shares Distribution (5+ MPG):
count    155588.000000
mean          0.192451
std           0.886852
min          -7.312516
25%          -0.102330
50%           0.057222
75%           0.514040
max           8.972702
Name: win_shares, dtype: float64

Offensive Win Shares:
count    155588.000000
mean          0.084897
std           0.902536
min         -10.213922
25%          -0.212579
50%           0.000000
75%           0.385870
max           8.906983
Name: offensive_win_shares, dtype: float64

Defensive Win Shares:
count    155588.000000
mean          0.107554
std           0.345278
min          -5.339197
25%           0.000000
50%           0.051620
75%           0.204145
max           6.750110
Name: defensive_win_shares, dtype: float64

Top 10 by Win Shares:
       full_name         team  season  week week_start  team_wins  games_played_this_week       mpg       ppg  offensive_win_shares  defensive_win_shares 

In [16]:
list(features_overall_weekly_df.columns)

['player_id',
 'full_name',
 'team',
 'season',
 'week',
 'week_start',
 'conference',
 'pow_conference',
 'games_played_this_week',
 'numMinutes',
 'points',
 'assists',
 'blocks',
 'steals',
 'reboundsTotal',
 'reboundsDefensive',
 'reboundsOffensive',
 'fieldGoalsAttempted',
 'fieldGoalsMade',
 'threePointersAttempted',
 'threePointersMade',
 'freeThrowsAttempted',
 'freeThrowsMade',
 'turnovers',
 'foulsPersonal',
 'plusMinusPoints',
 'wins_this_week',
 'wins_vs_team_with_all_nba_player',
 'is_win_vs_over_500',
 'opponent_has_all_nba',
 'avg_opp_score',
 'avg_opp_winrate_prior',
 'avg_opp_wins_prior',
 'avg_opp_losses_prior',
 'away_games_prior',
 'away_losses_prior',
 'away_win_streak_prior',
 'away_wins_prior',
 'home_games_prior',
 'home_losses_prior',
 'home_win_streak_prior',
 'home_wins_prior',
 'losses_prior',
 'wins_vs_over_500_prior',
 'won_player_of_the_week',
 'all_star_this_season',
 'mvp_this_season',
 'all_nba_first_team_this_season',
 'all_nba_second_team_this_season

In [17]:
features_overall_weekly_df = features_overall_weekly_df[
['player_id',
 'full_name',
 'team',
 'season',
 'week',
 'week_start',
 'conference',
 'pow_conference',
 'games_played_this_week',
 'numMinutes',
 'points',
 'assists',
 'blocks',
 'steals',
 'reboundsTotal',
 'reboundsDefensive',
 'reboundsOffensive',
 'fieldGoalsAttempted',
 'fieldGoalsMade',
 'threePointersAttempted',
 'threePointersMade',
 'freeThrowsAttempted',
 'freeThrowsMade',
 'turnovers',
 'foulsPersonal',
 'plusMinusPoints',
 'wins_this_week',
 'wins_vs_team_with_all_nba_player',
 'is_win_vs_over_500',
 'opponent_has_all_nba',
 'avg_opp_score',
 'avg_opp_winrate_prior',
 'avg_opp_wins_prior',
 'avg_opp_losses_prior',
 'away_games_prior',
 'away_losses_prior',
 'away_win_streak_prior',
 'away_wins_prior',
 'home_games_prior',
 'home_losses_prior',
 'home_win_streak_prior',
 'home_wins_prior',
 'losses_prior',
 'wins_vs_over_500_prior',
 'won_player_of_the_week',
 'all_star_this_season',
 'mvp_this_season',
 'all_nba_first_team_this_season',
 'all_nba_second_team_this_season',
 'all_nba_third_team_this_season',
 'team_ast',
 'team_blk',
 'team_stl',
 'team_gms',
 'pow_player_id',
 'player_of_the_week',
 'fieldGoalsPercentage',
 'threePointersPercentage',
 'freeThrowsPercentage',
 'points_mean_season',
 'points_std_season',
 'assists_mean_season',
 'assists_std_season',
 'plusMinusPoints_mean_season',
 'plusMinusPoints_std_season',
 'z_pts',
 'z_ast',
 'z_pm',
 'breakout_score',
 'team_fga',
 'opp_team_fga',
 'team_pts',
 'opp_team_pts',
 'team_assists',
 'team_fg',
 'team_ppg',
 'lg_ppg',
 'player_efficiency_rating',
#  'opp_ppg',
#  'mpg',
#  'wins_this_week_clean',
#  'ppg',
#  'player_poss',
#  'team_poss',
#  'opp_poss',
#  'ortg',
#  'marginal_offense',
#  'ows',
#  'team_drtg',
#  'team_min_week',
#  'min_pct',
#  'stops',
#  'stop_rate',
#  'drtg',
#  'marginal_defense',
#  'dws',
#  'ws_unscaled',
#  'scale_factor',
 'win_shares',
 'offensive_win_shares',
 'defensive_win_shares',
#  'team_off_contrib',
#  'team_def_contrib',
#  'team_total_contrib',
#  'team_wins',
#  'abs_contrib',
#  'team_abs_contrib',
#  'contrib_share',
#  'ws_magnitude',
#  'ws_signed',
#  'off_share',
#  'ws_marginal',
#  'team_marginal_ws',
#  'team_ows',
#  'team_dws',
#  'team_wins_drop',
#  'ws_scale',
#  'ws_scale_capped'
 ]
]
features_overall_weekly_df.head()

,player_id,full_name,team,season,week,week_start,conference,pow_conference,games_played_this_week,numMinutes,...,team_pts,opp_team_pts,team_assists,team_fg,team_ppg,lg_ppg,player_efficiency_rating,win_shares,offensive_win_shares,defensive_win_shares
0,1453,Walter Davis,Suns,1979,7,1980-02-11,West,West,3,84.0,...,336.166503,336.148597,90.0,145.0,112.055501,64.708926,13.107539,-0.065397,-0.095677,0.030280
1,76003,Kareem Abdul-Jabbar,Lakers,1979,7,1980-02-11,West,West,3,106.0,...,243.000000,200.000000,102.0,145.0,81.000000,64.708926,29.561685,1.062230,0.627513,0.434717
2,76011,Alvan Adams,Suns,1979,7,1980-02-11,West,West,3,92.0,...,336.166503,336.148597,90.0,145.0,112.055501,64.708926,16.067036,0.357255,0.340735,0.016520
3,76229,Alonzo Bradley,Rockets,1979,7,1980-02-11,West,West,2,20.0,...,219.000000,230.000000,98.0,175.0,109.500000,64.708926,-4.879795,0.818714,0.812902,0.005812
4,76242,Mike Bratz,Suns,1979,7,1980-02-11,West,West,3,54.0,...,336.166503,336.148597,90.0,145.0,112.055501,64.708926,12.392648,-1.195374,-1.163923,-0.031452


In [18]:
# Drop columns that contain the answer. Also drop mvp, all-star, all-nba because they are from the future when we try to do inference during season
leakage_cols = ['pow_player_id', 'player_of_the_week', 'pow_conference',
                'all_star_this_season', 'mvp_this_season',
                'all_nba_first_team_this_season', 'all_nba_second_team_this_season',
                'all_nba_third_team_this_season']

# Set aside identifier columns
id_cols = ['player_id', 'full_name', 'team', 'season', 'week', 'week_start', 'conference']

features_overall_weekly_df_clean = features_overall_weekly_df.drop(columns=leakage_cols + id_cols)
df_encoded = pd.get_dummies(features_overall_weekly_df_clean, drop_first=True)
df_encoded

,games_played_this_week,numMinutes,points,assists,blocks,steals,reboundsTotal,reboundsDefensive,reboundsOffensive,fieldGoalsAttempted,...,team_pts,opp_team_pts,team_assists,team_fg,team_ppg,lg_ppg,player_efficiency_rating,win_shares,offensive_win_shares,defensive_win_shares
0,3,84.00,50.0,17.0,0.0,1.0,13.0,2.0,1.0,36.0,...,336.166503,336.148597,90.0,145.0,112.055501,64.708926,13.107539,-0.065397,-0.095677,0.030280
1,3,106.00,83.0,19.0,16.0,6.0,21.0,14.0,7.0,51.0,...,243.000000,200.000000,102.0,145.0,81.000000,64.708926,29.561685,1.062230,0.627513,0.434717
2,3,92.00,53.0,10.0,7.0,4.0,28.0,4.0,2.0,40.0,...,336.166503,336.148597,90.0,145.0,112.055501,64.708926,16.067036,0.357255,0.340735,0.016520
3,2,20.00,5.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,...,219.000000,230.000000,98.0,175.0,109.500000,64.708926,-4.879795,0.818714,0.812902,0.005812
4,3,54.00,31.0,12.0,1.0,4.0,5.0,1.0,1.0,29.0,...,336.166503,336.148597,90.0,145.0,112.055501,64.708926,12.392648,-1.195374,-1.163923,-0.031452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159432,4,80.23,47.0,4.0,2.0,3.0,11.0,10.0,1.0,29.0,...,441.000000,445.000000,107.0,150.0,110.250000,114.371027,25.405111,-1.731877,-2.357656,0.625779
159433,3,80.39,13.0,10.0,1.0,1.0,21.0,14.0,7.0,21.0,...,292.000000,369.000000,79.0,110.0,97.333333,114.371027,7.909301,0.000000,0.000000,0.000000
159434,3,136.54,38.0,8.0,0.0,2.0,16.0,14.0,2.0,44.0,...,329.000000,375.000000,150.0,242.0,109.666667,114.371027,6.756053,0.000000,0.000000,0.000000
159435,3,83.90,33.0,4.0,0.0,3.0,6.0,6.0,0.0,34.0,...,292.000000,369.000000,79.0,110.0,97.333333,114.371027,9.873812,0.000000,0.000000,0.000000


In [19]:
from sklearn.model_selection import GroupShuffleSplit

X = df_encoded.drop(columns=['won_player_of_the_week'])
y = df_encoded['won_player_of_the_week']
identifiers = features_overall_weekly_df[id_cols]

# Create groups for each season-week-conference combination
groups = (features_overall_weekly_df['season'].astype(str) + '_' +
          features_overall_weekly_df['week'].astype(str) + '_' +
          features_overall_weekly_df['conference'].astype(str))

# Use GroupShuffleSplit instead of train_test_split
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

# Split using the indices
X_train = X.iloc[train_idx]
X_test = X.iloc[test_idx]
y_train = y.iloc[train_idx]
y_test = y.iloc[test_idx]
id_train = identifiers.iloc[train_idx]
id_test = identifiers.iloc[test_idx]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Positive class in y_train: {100 * round(y_train.mean(),4)}%")
print(f"Positive class in y_test: {100 * round(y_test.mean(),4)}%")

Training samples: 127385
Test samples: 32052
Positive class in y_train: 0.7100000000000001%
Positive class in y_test: 0.72%


In [20]:
!pip install lightgbm

In [21]:
import lightgbm as lgb

In [22]:
# Train LightGBM model
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    num_leaves=31,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# Train LightGBM model
lgb_model.fit(X_train, y_train)

# Get predictions
y_pred_proba = lgb_model.predict_proba(X_test)[:, 1]

In [24]:
evaluation_df = id_test.copy()
evaluation_df['actual_won'] = y_test.values
evaluation_df['pred_proba'] = y_pred_proba
evaluation_df

,player_id,full_name,team,season,week,week_start,conference,actual_won,pred_proba
225,229,James Edwards,Pacers,1980,47,1980-11-17,East,0,0.000447
226,305,Robert Parish,Celtics,1980,47,1980-11-17,East,0,0.475091
227,328,Rick Mahorn,Bullets,1980,47,1980-11-17,East,0,0.000083
228,940,Earl Cureton,76ers,1980,47,1980-11-17,East,0,0.000036
229,1449,Larry Bird,Celtics,1980,47,1980-11-17,East,0,0.025370
...,...,...,...,...,...,...,...,...,...
159432,1642366,Quinten Post,Warriors,2024,12,2025-03-17,West,0,0.000124
159433,1642367,Jonathan Mogbo,Raptors,2024,12,2025-03-17,East,0,0.000056
159434,1642377,Jaylen Wells,Grizzlies,2024,12,2025-03-17,West,0,0.000043
159435,1642419,Jamison Battle,Raptors,2024,12,2025-03-17,East,0,0.000043


In [25]:
actual_winners_df = evaluation_df[evaluation_df['actual_won'] == 1]
actual_winners_df.head()

,player_id,full_name,team,season,week,week_start,conference,actual_won,pred_proba
289,600014,Artis Gilmore,Bulls,1980,47,1980-11-17,East,1,0.176648
570,76003,Kareem Abdul-Jabbar,Lakers,1983,6,1984-02-06,West,1,0.807943
747,787,Charles Barkley,76ers,1985,13,1986-03-24,East,1,0.933420
1230,938,Rony Seikaly,Heat,1989,10,1990-03-05,East,1,0.267551
1339,893,Michael Jordan,Bulls,1989,49,1989-12-04,East,1,0.960137


In [26]:
print(f"Total samples in evaluation_df: {len(evaluation_df)}")
print(f"Actual winners: {len(actual_winners_df)}")

Total samples in evaluation_df: 32052
Actual winners: 230


In [27]:


# How many season,conference,week combinations were there in actual_winners_df? This is our denominator for accuracy.

query = """
SELECT COUNT(DISTINCT(season,conference,week)) AS total_season_conference_weeks_we_couldve_predicted
FROM actual_winners_df
"""

total_season_conference_weeks_we_couldve_predicted = duckdb.query(query).df()['total_season_conference_weeks_we_couldve_predicted'].sum()
total_season_conference_weeks_we_couldve_predicted

# How many of the actual winners did we predict?
# Aka how many of the actual winners did our model assign a top-k probability for that [season,conference,week] combo?
for k in [1, 2, 3, 4, 5, 10]:
  query = f"""

  WITH Highest_Probabilities AS (
  SELECT *
  ,RANK() OVER(PARTITION BY season, conference, week ORDER BY pred_proba DESC) AS rnk
  FROM evaluation_df
  )
  ,
  Actual_Winners AS (
    SELECT * FROM evaluation_df
    WHERE actual_won = 1
  )


  SELECT * FROM Highest_Probabilities
  JOIN Actual_Winners
  ON
  Highest_Probabilities.season = Actual_Winners.season
  AND
  Highest_Probabilities.week = Actual_Winners.week
  AND
  Highest_Probabilities.conference = Actual_Winners.conference
  WHERE rnk <= {k}
  AND Highest_Probabilities.pred_proba = Actual_Winners.pred_proba
  """
  duckdb.query(query).df()
  correct_predictions_df = duckdb.query(query).df()
  total_correct_predictions = correct_predictions_df["actual_won"].sum()
  print(f"Top-{k} accuracy:", round(100*total_correct_predictions/total_season_conference_weeks_we_couldve_predicted,2),"%")

Top-1 accuracy: 57.64 %
Top-2 accuracy: 79.48 %
Top-3 accuracy: 88.65 %
Top-4 accuracy: 93.01 %
Top-5 accuracy: 94.76 %
Top-10 accuracy: 98.69 %


In [28]:
feature_importance = pd.DataFrame({
  'feature': X_train.columns,
  'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

feature_importance['importance_pct'] = (feature_importance['importance'] / feature_importance['importance'].sum() * 100)

print("LightGBM Top 20 features:")
print(feature_importance.head(20)[['feature','importance_pct']])

LightGBM Top 20 features:
                     feature  importance_pct
61  player_efficiency_rating        5.833333
2                     points        5.500000
18            wins_this_week        4.366667
58                   team_fg        3.633333
60                    lg_ppg        3.000000
3                    assists        2.900000
40      fieldGoalsPercentage        2.666667
59                  team_ppg        2.633333
6              reboundsTotal        2.633333
43        points_mean_season        2.533333
4                     blocks        2.466667
7          reboundsDefensive        2.366667
10            fieldGoalsMade        2.333333
38                  team_stl        2.233333
45       assists_mean_season        2.133333
1                 numMinutes        2.133333
17           plusMinusPoints        1.966667
56              opp_team_pts        1.966667
50                     z_ast        1.833333
13       freeThrowsAttempted        1.800000


In [31]:
print("\nTraining final model on 100% of data for deployment...")
final_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    num_leaves=31,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

final_model.fit(X, y)  # ALL data

# # 5. Save the final model
# joblib.dump(final_model, 'lightgbm_potw_model.pkl')
# joblib.dump(X.columns.tolist(), 'lightgbm_model_features.pkl')


Training final model on 100% of data for deployment...


LGBMClassifier(class_weight='balanced', max_depth=6, n_jobs=-1, random_state=42,
               verbose=-1)

In [32]:
# Feature importance from final model
final_feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

final_feature_importance['importance_pct'] = (final_feature_importance['importance'] / final_feature_importance['importance'].sum() * 100)

print("Final Model Top 20 features:")
print(final_feature_importance.head(20)[['feature','importance_pct']])

Final Model Top 20 features:
                     feature  importance_pct
2                     points        7.011686
61  player_efficiency_rating        6.176962
18            wins_this_week        4.507513
58                   team_fg        4.040067
3                    assists        3.005008
60                    lg_ppg        2.871452
1                 numMinutes        2.637730
40      fieldGoalsPercentage        2.604341
6              reboundsTotal        2.604341
4                     blocks        2.337229
38                  team_stl        2.270451
59                  team_ppg        2.270451
10            fieldGoalsMade        2.203673
7          reboundsDefensive        2.136895
43        points_mean_season        2.136895
49                     z_pts        2.070117
62                win_shares        2.003339
64      defensive_win_shares        1.969950
13       freeThrowsAttempted        1.969950
17           plusMinusPoints        1.769616
